In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.linear_model import (Lasso, Ridge, ElasticNet, BayesianRidge,
                                  HuberRegressor, LinearRegression)
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              HistGradientBoostingRegressor, AdaBoostRegressor,
                              BaggingRegressor, VotingRegressor)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.feature_selection import (RFE, mutual_info_regression, f_regression,
                                        SelectKBest, SelectFromModel, f_classif)
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import pearsonr
import os
import warnings
import pickle
import joblib
from joblib import Parallel, delayed
import multiprocessing
import random
import time
import hashlib  # 用于数据一致性检查

warnings.filterwarnings('ignore')

# 获取CPU核心数
n_cores = multiprocessing.cpu_count()
print(f"检测到 {n_cores} 个CPU核心，将使用并行计算")

# ============== 配置参数 ==============
CONFIG = {
    'data_file': "D:\\文成数据库\\Nb-Si数据库3.4-成分-性能.xlsx",
    'output_dir': "D:\\博一下\\manuscript5-变Si组\\模型训练_K4567_9.3",
    'target_col': 'KQ',
    'exclude_cols': ['ID', 'Sample_Name', 'Notes'],  # 需要排除的列
    'forbidden_features_for_final_model': ['Nb', 'Si', 'Ti', 'Al', 'Cr', 'Hf', 'Zr', 'Mo', 'V', 'W', 'Ta', 'Sn'],  # 最终模型中禁止的元素特征
    'max_k': 40,  # K的最大值限制（修改为40）
    'target_k_range': [4, 5, 6, 7],  # 处理K=4,5,6,7
    'min_r2_threshold': 0.905,  # 目标R²下限阈值
    'max_r2_threshold': 0.99,  # 目标R²上限阈值（避免过拟合）
    'cv_folds': 5,  # 交叉验证折数改为5折
    'random_state': 2023,
    'n_jobs': 4,  # 使用所有可用的CPU核心
    # 新增高级方法的参数
    'sfs_max_iter': 20,    # SFS最大迭代次数
    'sbs_max_iter': 20,    # SBS最大迭代次数
    'ga_population': 20,   # GA种群大小
    'ga_generations': 15,  # GA迭代代数
    'ga_mutation_rate': 0.1,  # GA变异率
    'ga_crossover_rate': 0.8,  # GA交叉率
    # 物理引导特征工程参数
    'physics_guided_enhancement': True,
    'exact_feature_names': True,           # 使用精确的特征名
    'auto_feature_detection': True,        # 自动检测可用特征
    'quality_check_enabled': True,         # 启用质量检查
    'correlation_threshold': 0.001,        # 极低相关性阈值
    'variance_threshold': 1e-12,          # 极小方差阈值
    # 贝叶斯优化参数 - 已禁用
    'bayesian_optimization': False,        # 禁用贝叶斯优化
    'print_all_satisfactory': True,       # 打印所有满足条件的模型
    # 新增：调试和文件保存参数
    'save_detailed_predictions': True,    # 保存详细预测结果
    'enable_data_consistency_check': True, # 启用数据一致性检查
    'debug_mode': True,                    # 启用调试模式
}

# 重点优化策略（保留用于统计，但不再进行贝叶斯优化）
FOCUS_MODELS = ['KNeighbors', 'GradientBoosting', 'XGBoost', 'CatBoost', 'RandomForest', 'AdaBoost', 'Bagging']
FOCUS_METHODS = ['SFS', 'GA', 'RandomForest', 'F-Regression', 'PCC']

# ============== 特征名称简称映射字典 ==============
def create_feature_name_mapping():
    """根据PDF文档创建特征名称简称映射"""
    mapping = {
        # 成分特征 (CC)
        'Nb': 'CC1',
        'Si': 'CC2', 
        'Ti': 'CC3',
        'Hf': 'CC4',
        'Zr': 'CC5',
        'Al': 'CC6',
        'Cr': 'CC7',
        'Mo': 'CC8',
        'V': 'CC9',
        'W': 'CC10',
        'Ta': 'CC11',
        'Sn': 'CC12',
        
        # 原子电子性质 (AE)
        'mean_A1 atomic number': 'AE1m',
        'var_A1 atomic number': 'AE1v',
        'mean_E2 electronegativity (Pauling)': 'AE2m',
        'var_E2 electronegativity (Pauling)': 'AE2v',
        'mean_Electrophilicity Index': 'AE3m',
        'var_Electrophilicity Index': 'AE3v',
        'mean_F13 Electron affinity': 'AE4m',
        'var_F13 Electron affinity': 'AE4v',
        'mean_Glawe Number': 'AE5m',
        'var_Glawe Number': 'AE5v',
        
        # 原子半径 (AR) - 根据文档推断
        'mean_S13 radii atomic (coordination number 12) (pm)': 'AR1m',
        'var_S13 radii atomic (coordination number 12) (pm)': 'AR1v',
        
        # 原子磁性 (AM) - 根据文档推断
        'mean_Mass Magnetic Susceptibility': 'AM1m',
        'var_Mass Magnetic Susceptibility': 'AM1v',
        
        # 原子核 (AN) - 根据文档推断
        'mean_A2 atomic mass': 'AN1m',
        'var_A2 atomic mass': 'AN1v',
        
        # 原子重量 (AW) - 根据文档推断
        'mean_A3 atomic weight': 'AW1m',
        'var_A3 atomic weight': 'AW1v',
        
        # 热力学 (TH) - 根据文档推断
        'mean_C1 temperature melting': 'TH1m',
        'var_C1 temperature melting': 'TH1v',
        'mean_C2 temperature boiling': 'TH2m',
        'var_C2 temperature boiling': 'TH2v',
        'mean_C4 enthalpy melting': 'TH3m',
        'var_C4 enthalpy melting': 'TH3v',
        'mean_C5 enthalpy atomization': 'TH4m',
        'var_C5 enthalpy atomization': 'TH4v',
        
        # 机械性能 (PM) - 根据文档推断
        'mean_C12 modulus Young': 'PM1m',
        'var_C12 modulus Young': 'PM1v',
        
        # 热学性能 (PT) - 根据文档推断
        'mean_C7 thermal conductivity': 'PT1m',
        'var_C7 thermal conductivity': 'PT1v',
        
        # 电学性能 (PE) - 根据文档推断
        'mean_E1 electrical resistivity': 'PE1m',
        'var_E1 electrical resistivity': 'PE1v',
        
        # 密度性能 (PD) - 根据文档推断
        'mean_密度': 'PD1m',
        'var_密度': 'PD1v',
        
        # 晶格结构 (SL) - 根据文档推断
        'mean_S10 Lattice Constants a': 'SL1m',
        'var_S10 Lattice Constants a': 'SL1v',
        'mean_S11 Lattice Constants b': 'SL2m',
        'var_S11 Lattice Constants b': 'SL2v',
        'mean_S12 Lattice Constants c': 'SL3m',
        'var_S12 Lattice Constants c': 'SL3v',
        
        # 空间群 (SS) - 根据文档推断
        'mean_S14 space group number': 'SS1m',
        'var_S14 space group number': 'SS1v',
        
        # 物理工程特征
        # 韧性判据 (JT)
        'Pugh_modulus_ratio': 'JT1',
        'Poisson_ratio_predicted': 'JT2', 
        'Cauchy_pressure_indicator': 'JT3',
        'Si_supersaturation': 'JT4',
        'atomic_size_mismatch_NbSi': 'JT5',
        
        # 电子结构判据 (JE)
        'VEC_Omega_coupling': 'JE1',
        'VEC_stabilization': 'JE2',
        'VEC_deviation_from_ideal': 'JE3',
        'electron_localization': 'JE4',
        
        # 热力学判据 (JH)
        'phase_stability_index': 'JH1',
        'thermodynamic_driving_force': 'JH2',
        'entropy_stabilization': 'JH3',
        'thermal_stability_factor': 'JH4',
        
        # Nb-Si专用判据 (JN)
        'solubility_limit_factor': 'JN1',
        'eutectic_distance': 'JN2',
        'Nb_Si_ratio': 'JN3',
        'silicide_volume_fraction': 'JN4',
        'hypereutectic_indicator': 'JN5',
        
        # 力学判据 (JM)
        'stress_concentration_factor': 'JM1',
        'lattice_distortion_energy': 'JM2',
        'toughness_indicator': 'JM3',
        'crack_resistance_factor': 'JM4',
        'plastic_deformation_capacity': 'JM5',
        
        # 扩散判据 (JD)
        'diffusion_resistance': 'JD1',
        'diffusion_activation_energy': 'JD2',
        
        # 合金判据 (JA)
        'refractory_strengthening': 'JA1',
        'Ti_Al_synergy': 'JA2',
        'Cr_oxidation_resistance': 'JA3',
        
        # 结构判据 (JS)
        'interface_coherency': 'JS1',
        'lattice_mismatch_effect': 'JS2',
        'phase_separation_tendency': 'JS3',
        'precipitation_hardening_potential': 'JS4',
        'elastic_anisotropy': 'JS5',
        
        # 其他判据 (JX)
        'enthalpy_entropy_competition': 'JX1',
        'silicide_forming_tendency': 'JX2',
        'VEC_squared_deviation': 'JX3',
        
        # 常见的热力学参数
        'VEC': 'VEC',
        'δ': 'Delta',
        'Ω': 'Omega',
        'ΔHmix': 'dHmix',
        'ΔSmix': 'dSmix',
        'ΔG': 'dG',
        'Tm': 'Tm',
        'ΔTm': 'dTm',
    }
    return mapping

def apply_feature_name_mapping(feature_names, mapping):
    """应用特征名称映射，将长名称转换为简称"""
    mapped_names = []
    for name in feature_names:
        if name in mapping:
            mapped_names.append(mapping[name])
        else:
            # 如果没有映射，保留原名称但进行简化
            simplified_name = name.replace('mean_', 'm_').replace('var_', 'v_')
            if len(simplified_name) > 15:
                simplified_name = simplified_name[:15] + '...'
            mapped_names.append(simplified_name)
    return mapped_names

# 创建输出目录
os.makedirs(CONFIG['output_dir'], exist_ok=True)

# 创建详细预测结果目录
predictions_dir = os.path.join(CONFIG['output_dir'], 'detailed_predictions')
os.makedirs(predictions_dir, exist_ok=True)

# ============== 新增：数据一致性检查函数 ==============
def calculate_data_checksum(data, label=""):
    """计算数据的校验和用于一致性检查"""
    if isinstance(data, np.ndarray):
        data_str = str(data.sum()) + str(data.shape) + str(data.mean())
    else:
        data_str = str(data)
    checksum = hashlib.md5(data_str.encode()).hexdigest()[:8]
    if CONFIG['debug_mode']:
        print(f"    🔍 {label} 数据校验和: {checksum}")
    return checksum

def verify_model_independence(model1, model2, model_name):
    """验证两个模型实例是否独立"""
    is_independent = id(model1) != id(model2)
    if CONFIG['debug_mode']:
        print(f"    🔍 模型独立性检查 {model_name}: {'✅ 独立' if is_independent else '❌ 相同实例'}")
    return is_independent

# ============== 新增：保存详细预测结果函数 ==============
def save_detailed_predictions(model_key, y_train, pred_train, y_test, pred_test, 
                            train_indices, test_indices, timestamp):
    """保存模型的详细预测结果到CSV文件"""
    if not CONFIG['save_detailed_predictions']:
        return
    
    # 创建训练集结果DataFrame
    train_results = pd.DataFrame({
        'Dataset': ['Training'] * len(y_train),
        'Dataset_Code': [1] * len(y_train),
        'Sample_Index': train_indices + 1,  # 从1开始编号
        'Experimental_KQ': y_train,
        'Predicted_KQ': pred_train,
        'Error': pred_train - y_train,
        'Absolute_Error': np.abs(pred_train - y_train),
        'Relative_Error_Percent': np.abs(pred_train - y_train) / np.abs(y_train) * 100
    })
    
    # 创建测试集结果DataFrame
    test_results = pd.DataFrame({
        'Dataset': ['Test'] * len(y_test),
        'Dataset_Code': [2] * len(y_test),
        'Sample_Index': test_indices + 1,  # 从1开始编号
        'Experimental_KQ': y_test,
        'Predicted_KQ': pred_test,
        'Error': pred_test - y_test,
        'Absolute_Error': np.abs(pred_test - y_test),
        'Relative_Error_Percent': np.abs(pred_test - y_test) / np.abs(y_test) * 100
    })
    
    # 合并训练集和测试集结果
    all_results = pd.concat([train_results, test_results], ignore_index=True)
    
    # 保存到唯一的文件名（包含时间戳避免覆盖）
    filename = f"{model_key}_predictions_{timestamp}.csv"
    filepath = os.path.join(predictions_dir, filename)
    
    all_results.to_csv(filepath, index=False, encoding='utf-8')
    
    if CONFIG['debug_mode']:
        print(f"    💾 详细预测结果已保存: {filename}")
        # 验证保存的数据
        saved_data = pd.read_csv(filepath)
        if len(saved_data) == len(all_results):
            print(f"    ✅ 文件保存验证成功，共{len(saved_data)}行数据")
        else:
            print(f"    ❌ 文件保存验证失败，期望{len(all_results)}行，实际{len(saved_data)}行")
    
    return filepath

# 定义RMSLE计算函数（处理负值情况）
def calculate_rmsle(y_true, y_pred):
    """
    计算Root Mean Squared Logarithmic Error
    处理负值：将负值转换为很小的正值
    """
    try:
        # 确保所有值都是正数，将负值和零转换为一个很小的正数
        y_true_pos = np.where(y_true <= 0, 1e-6, y_true)
        y_pred_pos = np.where(y_pred <= 0, 1e-6, y_pred)
        
        rmsle = np.sqrt(mean_squared_log_error(y_true_pos, y_pred_pos))
        return rmsle
    except:
        # 如果计算失败，返回NaN
        return np.nan

# 定义手动交叉验证函数（仅用于特征筛选阶段）
def manual_cross_val(model, X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=CONFIG['random_state'])
    r2_scores = []
    rmse_scores = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2_scores.append(r2)
        rmse_scores.append(rmse)
    return np.mean(r2_scores), np.std(r2_scores), np.mean(rmse_scores), np.std(rmse_scores)

# 定义快速评估函数（用于SFS、SBS、GA）
def quick_evaluate_features(X, y, feature_indices=None):
    """快速评估特征子集的性能"""
    if feature_indices is not None:
        if len(feature_indices) == 0:
            return -float('inf')
        X_subset = X[:, feature_indices]
    else:
        X_subset = X
    
    try:
        # 使用Ridge回归进行快速评估
        model = Ridge(alpha=1.0, random_state=CONFIG['random_state'])
        r2_mean, _, _, _ = manual_cross_val(model, X_subset, y, n_splits=3)  # 使用3折CV加速
        return r2_mean
    except:
        return -float('inf')

# ============== 基于实际数据库的物理引导特征工程实现 ==============

def physics_guided_feature_engineering_exact(X_reduced, remaining_features, y):
    """
    基于您实际数据库特征的物理引导特征工程
    使用确切的列名进行计算
    
    Args:
        X_reduced: 递归消除后的特征矩阵
        remaining_features: 剩余的特征名
        y: 目标变量
    
    Returns:
        X_enhanced: 增强后的特征矩阵
        enhanced_features: 增强后的特征名列表
    """
    
    print("=== 基于实际数据库的物理引导特征工程 ===")
    print(f"  基础特征数: {len(remaining_features)}")
    print(f"  数据库特征验证: 检查必要特征是否存在")
    
    # 将numpy数组转换为DataFrame
    feature_df = pd.DataFrame(X_reduced, columns=remaining_features)
    
    # 存储新特征
    new_features = {}
    
    # ============== 特征存在性检查函数 ==============
    def check_feature_exists(feature_name, feature_list):
        """检查特征是否存在"""
        return feature_name in feature_list
    
    def get_feature_safely(df, feature_name, default_value=0):
        """安全获取特征值"""
        if feature_name in df.columns:
            return df[feature_name]
        else:
            print(f"    ⚠️ 特征 '{feature_name}' 不存在，使用默认值")
            return pd.Series([default_value] * len(df), index=df.index)
    
    # ============== 第一类：核心韧性判据特征 ==============
    print("\n  === 核心韧性判据特征 ===")
    
    # 1. Pugh模量比 (B/G) - 基于VEC计算
    if check_feature_exists('VEC', remaining_features):
        new_features['Pugh_modulus_ratio'] = (feature_df['VEC'] * 9.4 - 4.0) / (feature_df['VEC'] * 4.2 + 25)
        print("  ✅ Pugh模量比 (B/G) - 基于VEC计算")
    else:
        print("  ❌ 缺少VEC，无法计算Pugh模量比")
    
    # 2. 泊松比预测 - 基于VEC
    if check_feature_exists('VEC', remaining_features):
        new_features['Poisson_ratio_predicted'] = 0.33 - 0.015 * feature_df['VEC']
        print("  ✅ 预测泊松比 - 基于VEC计算")
    
    # 3. Cauchy压力指标 - 基于VEC
    if check_feature_exists('VEC', remaining_features):
        new_features['Cauchy_pressure_indicator'] = -5.2 * feature_df['VEC'] + 45
        print("  ✅ Cauchy压力指标 - 基于VEC计算")
    
    # 4. Si过饱和度
    if check_feature_exists('Si', remaining_features):
        new_features['Si_supersaturation'] = np.maximum(0, feature_df['Si'] - 0.08)
        print("  ✅ Si过饱和度 - 基于Si含量计算")
    else:
        print("  ❌ 缺少Si含量，无法计算Si过饱和度")
    
    # 5. Nb-Si原子尺寸失配
    if check_feature_exists('Nb', remaining_features) and check_feature_exists('Si', remaining_features):
        # 基于Nb(1.43Å)和Si(1.11Å)原子半径
        r_Nb, r_Si = 1.43, 1.11
        new_features['atomic_size_mismatch_NbSi'] = (feature_df['Nb'] * feature_df['Si'] * 
                                                    ((r_Nb - r_Si) / r_Nb)**2)
        print("  ✅ Nb-Si原子尺寸失配 - 基于Nb、Si含量计算")
    else:
        print("  ❌ 缺少Nb或Si含量，无法计算原子尺寸失配")
    
    # ============== 第二类：电子结构特征 ==============
    print("\n  === 电子结构特征 ===")
    
    # 6. VEC-Ω耦合
    if (check_feature_exists('VEC', remaining_features) and 
        check_feature_exists('Ω', remaining_features)):
        new_features['VEC_Omega_coupling'] = feature_df['VEC'] * feature_df['Ω']
        print("  ✅ VEC-Ω耦合")
    
    # 7. VEC稳定化效应
    if (check_feature_exists('VEC', remaining_features) and 
        check_feature_exists('ΔHmix', remaining_features)):
        new_features['VEC_stabilization'] = feature_df['VEC'] / (1 + np.abs(feature_df['ΔHmix']))
        print("  ✅ VEC稳定化效应")
    
    # 8. VEC偏离度
    if check_feature_exists('VEC', remaining_features):
        new_features['VEC_deviation_from_ideal'] = np.abs(feature_df['VEC'] - 4.5)
        new_features['VEC_squared_deviation'] = (feature_df['VEC'] - 4.5)**2
        print("  ✅ VEC偏离度特征")
    
    # 9. 电子局域化参数
    if (check_feature_exists('mean_E2 electronegativity (Pauling)', remaining_features) and 
        check_feature_exists('var_E2 electronegativity (Pauling)', remaining_features)):
        mean_chi = feature_df['mean_E2 electronegativity (Pauling)']
        var_chi = feature_df['var_E2 electronegativity (Pauling)']
        new_features['electron_localization'] = mean_chi / (var_chi + 1e-6)
        print("  ✅ 电子局域化参数")
    
    # ============== 第三类：热力学稳定性特征 ==============
    print("\n  === 热力学稳定性特征 ===")
    
    R = 8.314  # 气体常数
    
    # 10. 热力学驱动力
    if (check_feature_exists('ΔG', remaining_features) and 
        check_feature_exists('Tm', remaining_features)):
        new_features['thermodynamic_driving_force'] = -feature_df['ΔG'] / (R * feature_df['Tm'])
        print("  ✅ 热力学驱动力")
    
    # 11. 熵稳定化效应
    if check_feature_exists('ΔSmix', remaining_features):
        new_features['entropy_stabilization'] = feature_df['ΔSmix'] / R
        print("  ✅ 熵稳定化效应")
    
    # 12. 焓熵竞争
    if (check_feature_exists('ΔHmix', remaining_features) and 
        check_feature_exists('ΔSmix', remaining_features) and 
        check_feature_exists('Tm', remaining_features)):
        new_features['enthalpy_entropy_competition'] = (np.abs(feature_df['ΔHmix']) / 
                                                       (feature_df['Tm'] * feature_df['ΔSmix'] + 1e-6))
        print("  ✅ 焓熵竞争效应")
    
    # 13. 相稳定性指数
    if (check_feature_exists('Ω', remaining_features) and 
        check_feature_exists('ΔHmix', remaining_features) and 
        check_feature_exists('Tm', remaining_features)):
        new_features['phase_stability_index'] = (feature_df['Ω'] * 
                                                np.exp(-np.abs(feature_df['ΔHmix'])/(R * feature_df['Tm'])))
        print("  ✅ 相稳定性指数")
    
    # 14. 热稳定性因子
    if (check_feature_exists('Tm', remaining_features) and 
        check_feature_exists('ΔTm', remaining_features)):
        new_features['thermal_stability_factor'] = feature_df['Tm'] / (np.abs(feature_df['ΔTm']) + 1)
        print("  ✅ 热稳定性因子")
    
    # ============== 第四类：Nb-Si体系专用特征 ==============
    print("\n  === Nb-Si体系专用特征 ===")
    
    # 15. Nb/Si比值
    if (check_feature_exists('Nb', remaining_features) and 
        check_feature_exists('Si', remaining_features)):
        new_features['Nb_Si_ratio'] = feature_df['Nb'] / (feature_df['Si'] + 1e-6)
        print("  ✅ Nb/Si比值")
    
    # 16. 共晶距离特征
    if check_feature_exists('Si', remaining_features):
        new_features['eutectic_distance'] = np.abs(feature_df['Si'] - 0.16)
        new_features['hypereutectic_indicator'] = np.maximum(0, feature_df['Si'] - 0.16)
        print("  ✅ 共晶距离和过共晶指示器")
    
    # 17. 硅化物形成倾向
    if (check_feature_exists('Si', remaining_features) and 
        check_feature_exists('mean_C5 enthalpy atomization', remaining_features)):
        new_features['silicide_forming_tendency'] = (feature_df['Si'] * 
                                                    feature_df['mean_C5 enthalpy atomization'])
        print("  ✅ 硅化物形成倾向")
    
    # 18. 硅化物体积分数预测
    if check_feature_exists('Si', remaining_features):
        new_features['silicide_volume_fraction'] = feature_df['Si'] * (1 - np.exp(-feature_df['Si'] / 0.1))
        print("  ✅ 硅化物体积分数预测")
    
    # 19. 固溶度限制因子
    if (check_feature_exists('Si', remaining_features) and 
        check_feature_exists('Tm', remaining_features)):
        new_features['solubility_limit_factor'] = feature_df['Si'] / (np.exp(-1000 / feature_df['Tm']) + 1e-6)
        print("  ✅ 固溶度限制因子")
    
    # ============== 第五类：韧性和力学性能特征 ==============
    print("\n  === 韧性和力学性能特征 ===")
    
    # 20. 韧性指示器
    if (check_feature_exists('mean_C12 modulus Young', remaining_features) and 
        check_feature_exists('mean_密度', remaining_features) and 
        check_feature_exists('var_E2 electronegativity (Pauling)', remaining_features)):
        E = feature_df['mean_C12 modulus Young']
        rho = feature_df['mean_密度']
        var_chi = feature_df['var_E2 electronegativity (Pauling)']
        new_features['toughness_indicator'] = np.sqrt(E * rho) / (var_chi + 1e-6)
        print("  ✅ 韧性指示器")
    
    # 21. 裂纹阻力因子
    if (check_feature_exists('mean_C12 modulus Young', remaining_features) and 
        check_feature_exists('mean_密度', remaining_features) and 
        check_feature_exists('var_S13 radii atomic (coordination number 12) (pm)', remaining_features)):
        E = feature_df['mean_C12 modulus Young']
        rho = feature_df['mean_密度']
        var_r = feature_df['var_S13 radii atomic (coordination number 12) (pm)']
        new_features['crack_resistance_factor'] = E / (rho * var_r + 1e-6)
        print("  ✅ 裂纹阻力因子")
    
    # 22. 弹性各向异性
    if (check_feature_exists('mean_C12 modulus Young', remaining_features) and 
        'Poisson_ratio_predicted' in new_features):
        E = feature_df['mean_C12 modulus Young']
        nu = new_features['Poisson_ratio_predicted']
        estimated_shear_modulus = E / (2 * (1 + nu))
        new_features['elastic_anisotropy'] = E / (estimated_shear_modulus + 1e-6)
        print("  ✅ 弹性各向异性参数")
    
    # 23. 塑性变形能力
    if ('Pugh_modulus_ratio' in new_features and 
        'Poisson_ratio_predicted' in new_features):
        new_features['plastic_deformation_capacity'] = (new_features['Pugh_modulus_ratio'] * 
                                                       new_features['Poisson_ratio_predicted'])
        print("  ✅ 塑性变形能力指标")
    
    # 24. 微观应力集中因子
    if (check_feature_exists('δ', remaining_features) and 
        check_feature_exists('var_S13 radii atomic (coordination number 12) (pm)', remaining_features)):
        new_features['stress_concentration_factor'] = (feature_df['δ'] * 
                                                      feature_df['var_S13 radii atomic (coordination number 12) (pm)'])
        print("  ✅ 微观应力集中因子")
    
    # ============== 第六类：扩散和动力学特征 ==============
    print("\n  === 扩散和动力学特征 ===")
    
    # 25. 扩散阻力
    if (check_feature_exists('mean_C1 temperature melting', remaining_features) and 
        check_feature_exists('mean_S13 radii atomic (coordination number 12) (pm)', remaining_features)):
        Tm = feature_df['mean_C1 temperature melting']
        r_atom = feature_df['mean_S13 radii atomic (coordination number 12) (pm)']
        new_features['diffusion_resistance'] = Tm / (r_atom**2 + 1e-6)
        print("  ✅ 扩散阻力")
    
    # 26. 扩散激活能估算
    if (check_feature_exists('mean_C1 temperature melting', remaining_features) and 
        check_feature_exists('ΔHmix', remaining_features)):
        Tm = feature_df['mean_C1 temperature melting']
        delta_H = feature_df['ΔHmix']
        new_features['diffusion_activation_energy'] = 0.3 * Tm * R + np.abs(delta_H)
        print("  ✅ 扩散激活能估算")
    
    # ============== 第七类：合金元素协同效应 ==============
    print("\n  === 合金元素协同效应 ===")
    
    # 27. 难熔元素强化效应
    refractory_elements = ['Ti', 'Hf', 'Zr', 'Mo', 'V', 'Ta']
    available_refractory = [elem for elem in refractory_elements if elem in remaining_features]
    
    if len(available_refractory) >= 2:
        refractory_sum = sum(feature_df[elem] for elem in available_refractory)
        new_features['refractory_elements_sum'] = refractory_sum
        
        if (check_feature_exists('mean_C1 temperature melting', remaining_features) and 
            check_feature_exists('mean_密度', remaining_features)):
            Tm = feature_df['mean_C1 temperature melting']
            rho = feature_df['mean_密度']
            new_features['refractory_strengthening'] = refractory_sum * Tm / rho
            print(f"  ✅ 难熔元素强化效应 (包含{len(available_refractory)}个元素: {available_refractory})")
    
    # 28. Ti-Al协同效应
    if (check_feature_exists('Ti', remaining_features) and 
        check_feature_exists('Al', remaining_features)):
        new_features['Ti_Al_synergy'] = feature_df['Ti'] * feature_df['Al'] * 2.0
        print("  ✅ Ti-Al协同效应")
    
    # 29. Cr抗氧化效应
    if (check_feature_exists('Cr', remaining_features) and 
        check_feature_exists('mean_E2 electronegativity (Pauling)', remaining_features) and 
        check_feature_exists('mean_S13 radii atomic (coordination number 12) (pm)', remaining_features)):
        Cr = feature_df['Cr']
        chi = feature_df['mean_E2 electronegativity (Pauling)']
        r_atom = feature_df['mean_S13 radii atomic (coordination number 12) (pm)']
        new_features['Cr_oxidation_resistance'] = Cr * chi / r_atom
        print("  ✅ Cr抗氧化效应")
    
    # ============== 第八类：界面和微观结构特征 ==============
    print("\n  === 界面和微观结构特征 ===")
    
    # 30. 界面共格度
    if check_feature_exists('var_S13 radii atomic (coordination number 12) (pm)', remaining_features):
        var_r = feature_df['var_S13 radii atomic (coordination number 12) (pm)']
        new_features['interface_coherency'] = 1 / (1 + var_r)
        print("  ✅ 界面共格度")
    
    # 31. 晶格失配效应
    if (check_feature_exists('var_S10 Lattice Constants a', remaining_features) and 
        check_feature_exists('var_S11 Lattice Constants b', remaining_features)):
        var_a = feature_df['var_S10 Lattice Constants a']
        var_b = feature_df['var_S11 Lattice Constants b']
        new_features['lattice_mismatch_effect'] = var_a * var_b
        print("  ✅ 晶格失配效应")
    
    # 32. 晶格畸变能
    if (check_feature_exists('δ', remaining_features) and 
        check_feature_exists('mean_C12 modulus Young', remaining_features)):
        delta = feature_df['δ']
        E = feature_df['mean_C12 modulus Young']
        new_features['lattice_distortion_energy'] = delta**2 * E
        print("  ✅ 晶格畸变能")
    
    # ============== 第九类：相分离和析出特征 ==============
    print("\n  === 相分离和析出特征 ===")
    
    # 33. 相分离倾向
    if (check_feature_exists('ΔHmix', remaining_features) and 
        check_feature_exists('ΔSmix', remaining_features)):
        new_features['phase_separation_tendency'] = feature_df['ΔHmix'] / (feature_df['ΔSmix'] + 1e-6)
        print("  ✅ 相分离倾向")
    
    # 34. 析出硬化潜力
    if (check_feature_exists('Si', remaining_features) and 
        check_feature_exists('mean_C12 modulus Young', remaining_features)):
        Si = feature_df['Si']
        E = feature_df['mean_C12 modulus Young']
        new_features['precipitation_hardening_potential'] = Si * E * 0.1
        print("  ✅ 析出硬化潜力")
    
    # ============== 质量检查和组合特征 ==============
    print(f"\n  === 特征质量检查 ===")
    print(f"  生成物理特征总数: {len(new_features)}个")
    
    if new_features:
        # 质量检查
        valid_features = {}
        correlation_scores = {}
        
        for feature_name, feature_values in new_features.items():
            # 数值稳定性检查
            if (not np.any(np.isnan(feature_values)) and 
                not np.any(np.isinf(feature_values)) and 
                np.var(feature_values) >= CONFIG['variance_threshold']):
                
                # 计算与目标变量的相关性
                try:
                    correlation = abs(np.corrcoef(feature_values, y)[0, 1])
                    if correlation >= CONFIG['correlation_threshold']:
                        valid_features[feature_name] = feature_values
                        correlation_scores[feature_name] = correlation
                except:
                    # 如果相关性计算失败，仍然保留特征
                    valid_features[feature_name] = feature_values
                    correlation_scores[feature_name] = CONFIG['correlation_threshold']
        
        print(f"  通过质量检查的特征: {len(valid_features)}个")
        
        # 按相关性排序显示
        if correlation_scores:
            sorted_features = sorted(correlation_scores.items(), key=lambda x: x[1], reverse=True)
            print(f"  相关性最高的前10个特征:")
            for i, (name, corr) in enumerate(sorted_features[:10]):
                print(f"    {i+1:2d}. {name}: r={corr:.4f}")
        
        if valid_features:
            # 组合原始特征和新特征
            valid_features_df = pd.DataFrame(valid_features, index=feature_df.index)
            enhanced_df = pd.concat([feature_df, valid_features_df], axis=1)
            
            # 特征类别统计
            print(f"\n  === 特征类别统计 ===")
            categories = {
                '核心韧性判据': ['Pugh_modulus_ratio', 'Poisson_ratio_predicted', 'Cauchy_pressure_indicator', 
                            'Si_supersaturation', 'atomic_size_mismatch_NbSi'],
                'Nb-Si专用': ['Nb_Si_ratio', 'eutectic_distance', 'hypereutectic_indicator', 
                            'silicide_forming_tendency', 'silicide_volume_fraction'],
                '电子结构': ['VEC_Omega_coupling', 'VEC_stabilization', 'VEC_deviation_from_ideal', 
                           'electron_localization'],
                '热力学': ['thermodynamic_driving_force', 'phase_stability_index', 'entropy_stabilization',
                         'thermal_stability_factor'],
                '力学性能': ['toughness_indicator', 'crack_resistance_factor', 'elastic_anisotropy',
                          'plastic_deformation_capacity'],
                '微观结构': ['interface_coherency', 'lattice_distortion_energy', 'precipitation_hardening_potential']
            }
            
            for category, feature_list in categories.items():
                count = sum(1 for f in feature_list if f in valid_features)
                if count > 0:
                    print(f"  {category}: {count}个特征")
            
            print(f"\n  ✅ 成功生成 {len(valid_features)} 个物理意义明确的特征")
            print(f"  ✅ 所有特征均基于您数据库中的现有特征计算")
            
            return enhanced_df.values, enhanced_df.columns.tolist()
        else:
            print("  ⚠️ 没有生成有效的物理特征")
            return X_reduced, remaining_features
    else:
        print("  ⚠️ 无法生成物理特征，可能缺少关键基础特征")
        return X_reduced, remaining_features

# ============== 1. 读取数据 ==============
print("=== 开始数据加载与预处理 ===")
df = pd.read_excel(CONFIG['data_file'])
print(f"✓ 成功加载数据: {df.shape}")

# 动态获取特征列名 - 排除目标列和指定的排除列
target_col = CONFIG['target_col']
exclude_cols = CONFIG['exclude_cols']

# 获取所有数值型特征列
all_columns = df.columns.tolist()
features_name = []

for col in all_columns:
    # 排除目标列和指定排除列
    if col != target_col and col not in exclude_cols:
        # 只保留数值型列
        if pd.api.types.is_numeric_dtype(df[col]):
            features_name.append(col)

print(f"✓ 动态识别到 {len(features_name)} 个数值特征")
print(f"✓ 目标列: {target_col}")
print(f"✓ 将在最终模型保存时过滤的元素特征: {CONFIG['forbidden_features_for_final_model']}")

# 检查目标列是否存在
if target_col not in df.columns:
    raise ValueError(f"目标列 '{target_col}' 不存在于数据中")

X_all = df[features_name]
y_all = df[[target_col]]  # DataFrame

# ============== 对数据进行标准化 ==============
print("\n=== 数据标准化与清理 ===")
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all)
y_all_np = y_all[target_col].values.ravel()

# ============== 移除 NaN ==============
# 检查并移除包含NaN的行
nan_mask_x = ~np.isnan(X_all_scaled).any(axis=1)
nan_mask_y = ~np.isnan(y_all_np)
valid_mask = nan_mask_x & nan_mask_y

X_all_scaled = X_all_scaled[valid_mask]
y_all_np = y_all_np[valid_mask]

# 保存原始数据用于树模型
X_all_original = X_all.values[valid_mask]

print(f"✓ 移除NaN后数据形状: X{X_all_scaled.shape}, y{y_all_np.shape}")
print(f"✓ 交叉验证设置: 仅在特征筛选阶段使用{CONFIG['cv_folds']}折交叉验证")

# 数据一致性检查
if CONFIG['enable_data_consistency_check']:
    print(f"\n=== 数据一致性检查 ===")
    calculate_data_checksum(X_all_scaled, "标准化特征矩阵")
    calculate_data_checksum(y_all_np, "目标变量")

# 保存初始特征名称列表
initial_features_name = features_name.copy()

# ============== 物理引导特征工程阶段 ==============
if CONFIG['physics_guided_enhancement']:
    print("\n=== 物理引导特征工程阶段 ===")
    X_all_scaled, features_name = physics_guided_feature_engineering_exact(
        X_all_scaled, features_name, y_all_np
    )
    
    # 同时更新原始数据矩阵（对于物理特征，使用标准化数据）
    if X_all_scaled.shape[1] > X_all_original.shape[1]:
        # 有新的物理特征被添加
        n_original_features = X_all_original.shape[1]
        n_new_features = X_all_scaled.shape[1] - n_original_features
        
        # 对于树模型，物理特征也使用标准化数据（因为物理特征是从标准化数据生成的）
        physics_features = X_all_scaled[:, n_original_features:]
        X_all_original = np.column_stack([X_all_original, physics_features])
    
    print(f"✓ 物理引导特征工程完成")
    
    # 数据一致性检查
    if CONFIG['enable_data_consistency_check']:
        calculate_data_checksum(X_all_scaled, "物理特征工程后的标准化特征矩阵")
else:
    print("\n=== 跳过物理引导特征工程 ===")

# ============== 第一阶段：相关性筛选冗余特征 ==============
print("\n=== 第一阶段: 相关性筛选 ===")
corr_matrix = np.corrcoef(X_all_scaled, rowvar=False)
remove_indices = []

for i in range(corr_matrix.shape[0]):
    for j in range(i + 1, corr_matrix.shape[1]):
        if abs(corr_matrix[i, j]) > 0.95:
            corr_i = abs(pearsonr(X_all_scaled[:, i], y_all_np)[0])
            corr_j = abs(pearsonr(X_all_scaled[:, j], y_all_np)[0])
            if corr_i < corr_j:
                remove_indices.append(i)
            else:
                remove_indices.append(j)

remove_indices = list(set(remove_indices))

# 找出要移除的特征名称
removed_features = [features_name[i] for i in remove_indices]
print(f"移除的高相关特征({len(removed_features)}个): {', '.join(removed_features[:10])}{'...' if len(removed_features) > 10 else ''}")

# 移除冗余特征
X_all_scaled = np.delete(X_all_scaled, remove_indices, axis=1)
X_all_original = np.delete(X_all_original, remove_indices, axis=1)
features_name = [name for i, name in enumerate(features_name) if i not in remove_indices]

print(f"✓ 第一阶段完成，剩余 {len(features_name)} 个特征")

# 数据一致性检查
if CONFIG['enable_data_consistency_check']:
    calculate_data_checksum(X_all_scaled, "相关性筛选后的特征矩阵")

# ============== 第二阶段：更稳健的递归消除（并行版本） ==============
print("\n=== 第二阶段: 递归消除 (使用稳健方法，并行计算) ===")

def evaluate_feature_subset(i, X, y, features_name):
    """评估移除第i个特征后的性能"""
    X_subset = np.delete(X, i, axis=1)
    
    # 检查子集是否有效
    if X_subset.shape[1] == 0:
        return i, float('inf')
    
    try:
        error_scores = []
        
        # 1. Ridge回归（处理多重共线性）
        ridge = Ridge(alpha=1.0, random_state=CONFIG['random_state'])
        ridge_scores = cross_val_score(ridge, X_subset, y, 
                                     scoring='neg_mean_squared_error', 
                                     cv=CONFIG['cv_folds'], 
                                     n_jobs=1)  # 子进程中不再并行
        if not np.any(np.isnan(ridge_scores)):
            error_scores.append(-np.mean(ridge_scores))
        
        # 2. Huber回归（对异常值稳健）
        huber = HuberRegressor(max_iter=200)
        huber_scores = cross_val_score(huber, X_subset, y, 
                                     scoring='neg_mean_squared_error', 
                                     cv=CONFIG['cv_folds'],
                                     n_jobs=1)
        if not np.any(np.isnan(huber_scores)):
            error_scores.append(-np.mean(huber_scores))
        
        # 3. 随机森林（非线性关系）
        rf = RandomForestRegressor(n_estimators=50, random_state=CONFIG['random_state'], n_jobs=1)
        rf_scores = cross_val_score(rf, X_subset, y, 
                                  scoring='neg_mean_squared_error', 
                                  cv=CONFIG['cv_folds'],
                                  n_jobs=1)
        if not np.any(np.isnan(rf_scores)):
            error_scores.append(-np.mean(rf_scores))
        
        # 如果至少有一个方法成功，使用平均误差
        if error_scores:
            error = np.mean(error_scores)
        else:
            error = float('inf')
        
    except Exception as e:
        error = float('inf')
    
    return i, error

def robust_recursive_elimination(X, X_orig, y, features_name):
    """使用更稳健的方法进行递归消除（并行版本）"""
    n = X.shape[1]
    last_error = float('inf')
    round_num = 1
    consecutive_no_improvement = 0
    
    print(f"  开始递归消除，当前特征数: {X.shape[1]}, 目标特征数: ≥{CONFIG['max_k']}")
    print(f"  使用 {CONFIG['n_jobs']} 个进程并行计算")
    
    while X.shape[1] > CONFIG['max_k']:
        print(f"\n  第 {round_num} 轮评估 (剩余特征: {X.shape[1]})")
        
        # 并行评估所有特征
        results = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1)(
            delayed(evaluate_feature_subset)(i, X, y, features_name) 
            for i in range(X.shape[1])
        )
        
        # 提取结果
        errors = [float('inf')] * X.shape[1]
        for i, error in results:
            errors[i] = error
        
        # 检查是否有有效的误差值
        valid_errors = [e for e in errors if e != float('inf')]
        if not valid_errors:
            print("  无法计算有效误差，停止递归消除")
            break
        
        # 找出移除后误差最小的特征
        min_error_index = np.argmin(errors)
        min_error = errors[min_error_index]
        
        # 如果最小误差是inf，说明没有有效的特征可以移除
        if min_error == float('inf'):
            print("  所有特征移除后都产生无效结果，停止递归消除")
            break
        
        print(f"  第 {round_num} 轮: 移除 '{features_name[min_error_index]}', 误差: {min_error:.4f}")
        
        # 检查是否改进
        if min_error >= last_error:
            consecutive_no_improvement += 1
            if consecutive_no_improvement >= 3:
                print("  连续3轮没有改进，停止递归消除")
                break
        else:
            consecutive_no_improvement = 0
        
        last_error = min_error
        X = np.delete(X, min_error_index, axis=1)
        X_orig = np.delete(X_orig, min_error_index, axis=1)
        features_name = [name for j, name in enumerate(features_name) if j != min_error_index]
        round_num += 1
        
        # 安全检查
        if X.shape[1] <= CONFIG['max_k']:
            print(f"  达到最大特征数限制 (K={CONFIG['max_k']})，停止递归消除")
            break
    
    return X, X_orig, features_name

# 执行稳健的递归消除
X_all_scaled, X_all_original, features_name = robust_recursive_elimination(
    X_all_scaled, X_all_original, y_all_np, features_name
)
print(f"✓ 第二阶段完成，剩余 {len(features_name)} 个特征")

# 数据一致性检查
if CONFIG['enable_data_consistency_check']:
    calculate_data_checksum(X_all_scaled, "递归消除后的特征矩阵")

# ============== 2. 定义特征选择方法（传统+高级方法） ==============

# 传统方法
def feature_selection_rfe(X, y, k):
    selector = RFE(estimator=Ridge(alpha=1.0), n_features_to_select=k)
    X_sel = selector.fit_transform(X, y)
    sel_indices = np.where(selector.support_)[0]
    return X[:, sel_indices], 'RFE', sel_indices

def feature_selection_mutual_info(X, y, k):
    selector = SelectKBest(mutual_info_regression, k=k)
    X_sel = selector.fit_transform(X, y)
    sel_indices = np.where(selector.get_support())[0]
    return X[:, sel_indices], 'Mutual Information', sel_indices

def feature_selection_f_regression(X, y, k):
    selector = SelectKBest(f_regression, k=k)
    X_sel = selector.fit_transform(X, y)
    sel_indices = np.where(selector.get_support())[0]
    return X[:, sel_indices], 'F-Regression', sel_indices

def feature_selection_pcc(X, y, k):
    pcc = [pearsonr(X[:, i], y)[0] for i in range(X.shape[1])]
    pcc = np.abs(pcc)  # Using absolute values of the correlation coefficients
    indices = np.argsort(pcc)[-k:]
    return X[:, indices], 'PCC', indices

def feature_selection_lasso(X, y, k):
    lasso = Lasso(alpha=0.01)
    lasso.fit(X, y)
    importance = np.abs(lasso.coef_)
    indices = np.argsort(importance)[-k:]
    return X[:, indices], 'Lasso', indices

def feature_selection_rf(X, y, k):
    rf = RandomForestRegressor(n_estimators=100, n_jobs=CONFIG['n_jobs'])
    rf.fit(X, y)
    importance = rf.feature_importances_
    indices = np.argsort(importance)[-k:]
    return X[:, indices], 'RandomForest', indices

def feature_selection_anova(X, y, k):
    f_values, p_values = f_classif(X, y)
    indices = np.argsort(f_values)[-k:]
    return X[:, indices], 'ANOVA_F', indices

def feature_selection_selectkbest(X, y, k):
    selector = SelectKBest(f_classif, k=k)
    X_sel = selector.fit_transform(X, y)
    sel_indices = np.where(selector.get_support())[0]
    return X[:, sel_indices], 'SelectKBest', sel_indices

# ============== 新增高级方法 ==============

def feature_selection_sfs(X, y, k):
    """
    Sequential Forward Selection (SFS) 顺序前向选择
    从空特征集开始，每次添加一个能最大化R²的特征
    """
    n_features = X.shape[1]
    selected_indices = []
    available_indices = list(range(n_features))
    
    print(f"    开始SFS，目标特征数: {k}, 可用特征数: {len(available_indices)}")
    
    for step in range(k):
        best_score = -float('inf')
        best_idx = None
        
        for idx in available_indices:
            # 临时特征集
            temp_indices = selected_indices + [idx]
            
            # 评估性能
            score = quick_evaluate_features(X, y, temp_indices)
            if score > best_score:
                best_score = score
                best_idx = idx
        
        if best_idx is not None:
            selected_indices.append(best_idx)
            available_indices.remove(best_idx)
            print(f"    SFS步骤{step+1}: 选择特征 '{features_name[best_idx]}', R²={best_score:.4f}")
        else:
            print(f"    SFS步骤{step+1}: 无可用特征")
            break
    
    if len(selected_indices) == k:
        return X[:, selected_indices], 'SFS', selected_indices
    else:
        print(f"    SFS未能选择足够特征: {len(selected_indices)}/{k}")
        return None, 'SFS', None

def feature_selection_sbs(X, y, k):
    """
    Sequential Backward Selection (SBS) 顺序后向选择
    从全特征集开始，每次移除一个对性能影响最小的特征
    """
    n_features = X.shape[1]
    selected_indices = list(range(n_features))
    
    print(f"    开始SBS，目标特征数: {k}, 初始特征数: {len(selected_indices)}")
    
    while len(selected_indices) > k:
        best_score = -float('inf')
        worst_idx = None
        
        for idx in selected_indices:
            # 临时移除该特征
            temp_indices = [i for i in selected_indices if i != idx]
            
            # 评估性能
            score = quick_evaluate_features(X, y, temp_indices)
            if score > best_score:
                best_score = score
                worst_idx = idx
        
        if worst_idx is not None:
            selected_indices.remove(worst_idx)
            step = len(selected_indices)
            print(f"    SBS步骤{n_features-step}: 移除特征 '{features_name[worst_idx]}', R²={best_score:.4f}")
        else:
            print("    SBS: 无法继续移除特征")
            break
    
    if len(selected_indices) == k:
        return X[:, selected_indices], 'SBS', selected_indices
    else:
        print(f"    SBS未能达到目标特征数: {len(selected_indices)}/{k}")
        return None, 'SBS', None

def feature_selection_ga(X, y, k):
    """
    Genetic Algorithm (GA) 遗传算法特征选择
    使用遗传算法搜索最优的k个特征组合
    """
    n_features = X.shape[1]
    
    print(f"    开始GA，目标特征数: {k}, 可用特征数: {n_features}")
    
    # 设置随机种子
    random.seed(CONFIG['random_state'])
    np.random.seed(CONFIG['random_state'])
    
    # 初始化种群
    population = []
    for _ in range(CONFIG['ga_population']):
        # 随机选择k个特征
        individual = random.sample(range(n_features), k)
        population.append(sorted(individual))
    
    def evaluate_individual(individual):
        """评估个体适应度"""
        return quick_evaluate_features(X, y, individual)
    
    def crossover(parent1, parent2):
        """交叉操作"""
        # 选择两个父代的特征组合，然后随机选择k个
        combined = list(set(parent1 + parent2))
        if len(combined) >= k:
            child = random.sample(combined, k)
        else:
            # 如果组合特征不足k个，补充随机特征
            child = combined.copy()
            remaining = [i for i in range(n_features) if i not in child]
            if len(remaining) > 0:
                child.extend(random.sample(remaining, min(k - len(child), len(remaining))))
        return sorted(child[:k])
    
    def mutate(individual):
        """变异操作"""
        if random.random() < CONFIG['ga_mutation_rate']:
            # 随机替换一个特征
            idx_to_replace = random.randint(0, len(individual) - 1)
            remaining = [i for i in range(n_features) if i not in individual]
            if remaining:
                individual[idx_to_replace] = random.choice(remaining)
        return sorted(individual)
    
    # 进化过程
    best_individual = None
    best_fitness = -float('inf')
    
    for generation in range(CONFIG['ga_generations']):
        # 评估适应度
        fitness_scores = []
        for individual in population:
            fitness = evaluate_individual(individual)
            fitness_scores.append(fitness)
            
            if fitness > best_fitness:
                best_fitness = fitness
                best_individual = individual.copy()
        
        if generation % 5 == 0:
            print(f"    GA第{generation}代: 最佳适应度={best_fitness:.4f}")
        
        # 选择操作（锦标赛选择）
        new_population = []
        
        # 保留最优个体
        new_population.append(best_individual.copy())
        
        while len(new_population) < CONFIG['ga_population']:
            # 锦标赛选择
            tournament_size = 3
            tournament_indices = random.sample(range(len(population)), tournament_size)
            tournament_fitness = [fitness_scores[i] for i in tournament_indices]
            winner_idx = tournament_indices[np.argmax(tournament_fitness)]
            parent1 = population[winner_idx]
            
            tournament_indices = random.sample(range(len(population)), tournament_size)
            tournament_fitness = [fitness_scores[i] for i in tournament_indices]
            winner_idx = tournament_indices[np.argmax(tournament_fitness)]
            parent2 = population[winner_idx]
            
            # 交叉和变异
            if random.random() < CONFIG['ga_crossover_rate']:
                child = crossover(parent1, parent2)
            else:
                child = parent1.copy()
            
            child = mutate(child)
            new_population.append(child)
        
        population = new_population
    
    print(f"    GA完成: 最佳适应度={best_fitness:.4f}")
    
    if best_individual is not None and len(best_individual) == k:
        selected_features = [features_name[i] for i in best_individual]
        print(f"    GA选择的特征: {', '.join(selected_features)}")
        return X[:, best_individual], 'GA', best_individual
    else:
        print(f"    GA未能找到有效解")
        return None, 'GA', None

# ============== 3. 特征筛选 - 处理K=4,5,6,7（传统+高级方法） ==============
print(f"\n=== 开始特征筛选 (处理K=4,5,6,7, 使用传统+高级特征选择方法+物理引导特征工程) ===")
print(f"传统方法: 8种 (RFE, 互信息, F回归, PCC, Lasso, 随机森林, ANOVA, SelectKBest)")
print(f"高级方法: 3种 (SFS, SBS, GA)")
print(f"物理引导: 基于实际数据库特征的物理意义特征工程")
print(f"筛选策略: 各方法直接选择最优特征，无需额外评估")
print(f"注意: 元素特征将参与特征选择，但包含指定元素特征的模型在最终保存时将被剔除")

good_feature_sets = []

# 处理K=4,5,6,7
for k in CONFIG['target_k_range']:
    # 检查是否有足够的特征
    if k > len(features_name):
        print(f"\n--- 特征数 K = {k} 超过可用特征数 {len(features_name)}，跳过 ---")
        continue
        
    print(f"\n--- 特征数 K = {k} ---")
    
    # 使用传统+高级特征选择方法
    feature_selectors = [
        # 传统方法
        feature_selection_rfe, 
        feature_selection_mutual_info, 
        feature_selection_f_regression,
        feature_selection_pcc, 
        feature_selection_lasso, 
        feature_selection_rf,
        feature_selection_anova, 
        feature_selection_selectkbest,
        # 高级方法
        feature_selection_sfs,
        feature_selection_sbs,
        feature_selection_ga
    ]

    for selector in feature_selectors:
        try:
            result = selector(X_all_scaled, y_all_np, k)
            if result[0] is None:
                print(f"{selector.__name__.replace('feature_selection_', '').upper():>25} | 跳过（特征选择失败）")
                continue
            X_sel, method_label, sel_idx = result
            X_sel_orig = X_all_original[:, sel_idx]
            
            # 数据一致性检查
            if CONFIG['enable_data_consistency_check']:
                calculate_data_checksum(X_sel, f"{method_label}_k={k} 选择的特征矩阵")
            
            # 直接加入特征集，不进行额外评估
            method_display = method_label
            if method_label in ['SFS', 'SBS', 'GA']:
                method_display = f"★ {method_label}"  # 标记高级方法
            
            print(f"{method_display:>25} | 成功选择 {k} 个特征")
            good_feature_sets.append((X_sel, X_sel_orig, 0.0, method_label, k, sel_idx))
            print(f"    ✓ 特征集已加入")
                
        except Exception as e:
            method_name = selector.__name__.replace('feature_selection_', '').upper()
            print(f"{method_name:>25} | 错误: {str(e)[:50]}")

print(f"\n=== 特征筛选完成 ===")
print(f"找到 {len(good_feature_sets)} 个特征集 (K=4,5,6,7, 传统+高级方法+物理引导特征工程)")
print(f"注意: 特征质量将通过后续的14种机器学习模型训练来评估")
print(f"注意: 所有模型均使用默认参数")

# ============== 4. 进行模型训练（使用默认参数，无贝叶斯优化） ==============
print(f"\n=== 开始模型训练与评估 (使用默认参数+物理引导特征工程+标准验证和多种评估指标: R², RMSE, MAE, MSE, RMSLE, 无贝叶斯优化) ===")
print(f"📊 标准验证: 80%-20%分割验证，评估多种指标")
print(f"⚡ 所有模型使用默认参数，无超参数优化")
print(f"🔍 调试模式已启用，将详细监控数据传递和模型独立性")

# 定义模型类字典（去除PCA、LightGBM和SVR）
models_classes = {
    "LinearRegression": LinearRegression,
    "Ridge": Ridge,
    "Lasso": Lasso,
    "ElasticNet": ElasticNet,
    "BayesianRidge": BayesianRidge,
    "HuberRegressor": HuberRegressor,
    "RandomForest": RandomForestRegressor,
    "GradientBoosting": GradientBoostingRegressor,
    "KNeighbors": KNeighborsRegressor,
    "DecisionTree": DecisionTreeRegressor,
    "XGBoost": XGBRegressor,
    "CatBoost": CatBoostRegressor,
    "HistGradientBoosting": HistGradientBoostingRegressor,
    "AdaBoost": AdaBoostRegressor,
    "Bagging": BaggingRegressor,
    "Voting": VotingRegressor
}

# 默认模型参数
def get_default_model(model_name, model_class):
    """获取带有合适默认参数的模型实例"""
    default_params = {}
    
    if 'random_state' in model_class().get_params():
        default_params['random_state'] = CONFIG['random_state']
    
    if 'n_jobs' in model_class().get_params() and model_name not in ['CatBoost']:
        default_params['n_jobs'] = CONFIG['n_jobs']
    elif model_name == 'CatBoost':
        default_params['verbose'] = 0
        default_params['random_state'] = CONFIG['random_state']
        if CONFIG['n_jobs'] > 0:
            default_params['thread_count'] = CONFIG['n_jobs']
        else:
            default_params['thread_count'] = -1
    
    # 特殊模型的额外参数
    if model_name == 'Voting':
        # 为VotingRegressor定义子估计器
        default_params = {
            'estimators': [
                ('rf', RandomForestRegressor(n_estimators=100, random_state=CONFIG['random_state'], n_jobs=CONFIG['n_jobs'])),
                ('knn', KNeighborsRegressor(n_jobs=CONFIG['n_jobs']))
            ],
            'n_jobs': CONFIG['n_jobs']
        }
    
    return model_class(**default_params)

# 判断是否使用树模型的原始数据
tree_based_models = ['RandomForest', 'GradientBoosting', 'XGBoost', 
                    'CatBoost', 'DecisionTree', 'HistGradientBoosting', 'AdaBoost', 'Bagging']

results_dict = {}
all_qualified_models = []  # 存储所有满足R²条件的模型

# 生成时间戳用于文件命名
timestamp = int(time.time())

for idx, (X_sel, X_sel_orig, r2_val, method_label, k, sel_idx) in enumerate(good_feature_sets):
    print(f"\n[{idx+1}/{len(good_feature_sets)}] 评估特征集: {method_label}_k={k}")
    
    # 打印选择的特征名称
    if sel_idx is not None:
        selected_features = [features_name[i] for i in sel_idx]
        print(f"  选择的特征: {', '.join(selected_features)}")
    else:
        print(f"  选择的特征: 特征转换方法 (无原始特征名)")
    
    # 特别标注高级方法和重点方法的结果
    if method_label in ['SFS', 'SBS', 'GA']:
        print(f"  ★ {method_label}高级方法选择的特征集")
    if method_label in FOCUS_METHODS:
        print(f"  🎯 {method_label}重点方法")
    
    # 数据一致性检查
    if CONFIG['enable_data_consistency_check']:
        calculate_data_checksum(X_sel, f"{method_label}_k={k} 标准化数据")
        calculate_data_checksum(X_sel_orig, f"{method_label}_k={k} 原始数据")
    
    feature_set_key = f"{method_label}_k={k}"
    
    for m_name, model_class in models_classes.items():
        try:
            # 根据模型类型选择数据
            if m_name in tree_based_models:
                # 树模型使用原始数据
                X_train_sel, X_test_sel, y_train, y_test = train_test_split(
                    X_sel_orig, y_all_np, test_size=0.2, random_state=CONFIG['random_state']
                )
                X_for_eval = X_sel_orig
                data_type = "原始数据"
            else:
                # 其他模型使用标准化数据
                X_train_sel, X_test_sel, y_train, y_test = train_test_split(
                    X_sel, y_all_np, test_size=0.2, random_state=CONFIG['random_state']
                )
                X_for_eval = X_sel
                data_type = "标准化数据"
            
            # 获取训练和测试索引
            _, _, _, _, train_indices, test_indices = train_test_split(
                X_for_eval, y_all_np, np.arange(len(y_all_np)), 
                test_size=0.2, random_state=CONFIG['random_state']
            )
            
            # 数据一致性检查
            if CONFIG['enable_data_consistency_check']:
                calculate_data_checksum(X_train_sel, f"{method_label}_{m_name} 训练数据({data_type})")
                calculate_data_checksum(X_test_sel, f"{method_label}_{m_name} 测试数据({data_type})")
            
            # 使用默认参数创建模型 - 确保每次都是新实例
            model = get_default_model(m_name, model_class)
            
            # 模型独立性检查
            if CONFIG['debug_mode'] and idx > 0:
                previous_model = get_default_model(m_name, model_class)
                verify_model_independence(model, previous_model, m_name)
            
            # 为重点模型和重点方法组合提供标记
            is_focus_combination = (m_name in FOCUS_MODELS and method_label in FOCUS_METHODS)
            if is_focus_combination:
                print(f"    🎯 重点组合: {method_label} + {m_name}")
            
            if CONFIG['debug_mode']:
                print(f"    🔍 使用{data_type}训练模型: {m_name}")
                print(f"    🔍 模型实例ID: {id(model)}")
            else:
                print(f"    使用默认参数: {m_name}")
            
            # 训练模型
            model.fit(X_train_sel, y_train)
            pred_train = model.predict(X_train_sel)
            pred_test = model.predict(X_test_sel)

            # 立即计算和验证指标
            r2_train = r2_score(y_train, pred_train)
            rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
            mae_train = mean_absolute_error(y_train, pred_train)
            mse_train = mean_squared_error(y_train, pred_train)
            rmsle_train = calculate_rmsle(y_train, pred_train)
            
            r2_test = r2_score(y_test, pred_test)
            rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))
            mae_test = mean_absolute_error(y_test, pred_test)
            mse_test = mean_squared_error(y_test, pred_test)
            rmsle_test = calculate_rmsle(y_test, pred_test)

            # 调试模式下打印详细信息
            if CONFIG['debug_mode']:
                print(f"    🔍 立即计算的指标: R²={r2_train:.4f}/{r2_test:.4f}, MAE={mae_train:.4f}/{mae_test:.4f}")
                print(f"    🔍 预测值范围: 训练集[{pred_train.min():.2f}, {pred_train.max():.2f}], 测试集[{pred_test.min():.2f}, {pred_test.max():.2f}]")

            # 保存结果到字典
            model_key = f"{method_label}_k={k}_{m_name}"
            results_dict[model_key] = {
                "Train R²": r2_train,
                "Test R²": r2_test,
                "Train RMSE": rmse_train,
                "Test RMSE": rmse_test,
                "Train MAE": mae_train,
                "Test MAE": mae_test,
                "Train MSE": mse_train,
                "Test MSE": mse_test,
                "Train RMSLE": rmsle_train,
                "Test RMSLE": rmsle_test,
                "Selected Indices": sel_idx,
                "K": k,
                "Optimized": False,  # 无优化
                "Focus Combination": is_focus_combination,
                "Data Type": data_type,  # 记录使用的数据类型
                "Model Instance ID": id(model)  # 记录模型实例ID
            }
            
            # 检查是否满足目标条件（避免过拟合：0.905 ≤ R² < 0.99）
            if (r2_train >= CONFIG['min_r2_threshold'] and r2_train < CONFIG['max_r2_threshold'] and 
                r2_test >= CONFIG['min_r2_threshold'] and r2_test < CONFIG['max_r2_threshold']):
                
                # 保存详细预测结果到CSV文件
                if CONFIG['save_detailed_predictions']:
                    saved_file = save_detailed_predictions(
                        model_key, y_train, pred_train, y_test, pred_test,
                        train_indices, test_indices, timestamp
                    )
                
                # 克隆模型以避免状态混乱
                from sklearn.base import clone
                model_copy = clone(model)
                model_copy.fit(X_for_eval, y_all_np)  # 使用全部数据重新训练
                
                all_qualified_models.append({
                    'model_key': model_key,
                    'method': method_label,
                    'model': m_name,
                    'k': k,
                    'train_r2': r2_train,
                    'test_r2': r2_test,
                    'train_mae': mae_train,
                    'test_mae': mae_test,
                    'train_mse': mse_train,
                    'test_mse': mse_test,
                    'train_rmsle': rmsle_train,
                    'test_rmsle': rmsle_test,
                    'features': [features_name[i] for i in sel_idx] if sel_idx is not None else 'Transformed',
                    'model_instance': model_copy,  # 保存重新训练的模型
                    'X_data': X_for_eval,            # 保存对应的数据
                    'X_sel': X_sel,                # 保存标准化数据
                    'X_sel_orig': X_sel_orig,      # 保存原始数据
                    'y_data': y_all_np,            # 保存目标数据
                    'sel_idx': sel_idx,
                    'is_tree_based': m_name in tree_based_models,  # 标记是否是树模型
                    'is_advanced': method_label in ['SFS', 'SBS', 'GA'],  # 标记是否是高级方法
                    'optimized': False,    # 标记未进行优化
                    'is_focus_model': m_name in FOCUS_MODELS,  # 标记是否是重点模型
                    'is_focus_method': method_label in FOCUS_METHODS,  # 标记是否是重点方法
                    'is_focus_combination': is_focus_combination,  # 标记是否是重点组合
                    'data_type': data_type,  # 记录使用的数据类型
                    'original_model_id': id(model),  # 记录原始模型ID
                    'predictions_file': saved_file if CONFIG['save_detailed_predictions'] else None,  # 记录预测文件路径
                })
                
                advanced_mark = "★" if method_label in ['SFS', 'SBS', 'GA'] else ""
                focus_mark = "🎯" if is_focus_combination else ""
                
                if method_label in ['SFS', 'SBS', 'GA']:
                    print(f"    {advanced_mark}★★ {method_label}+{m_name}{focus_mark}: "
                          f"R²={r2_train:.3f}/{r2_test:.3f}, MAE={mae_train:.3f}/{mae_test:.3f}, "
                          f"RMSLE={rmsle_train:.3f}/{rmsle_test:.3f} [满足目标!]")
                else:
                    print(f"    ★★★ {m_name}{focus_mark}: "
                          f"R²={r2_train:.3f}/{r2_test:.3f}, MAE={mae_train:.3f}/{mae_test:.3f}, "
                          f"RMSLE={rmsle_train:.3f}/{rmsle_test:.3f} [满足目标!]")
                    
            else:
                focus_mark = "🎯" if is_focus_combination else ""
                if method_label in ['SFS', 'SBS', 'GA']:
                    print(f"    · {method_label}+{m_name}{focus_mark}: R²={r2_train:.3f}/{r2_test:.3f}, MAE={mae_train:.3f}/{mae_test:.3f}, RMSLE={rmsle_train:.3f}/{rmsle_test:.3f}")
                else:
                    print(f"    · {m_name}{focus_mark}: R²={r2_train:.3f}/{r2_test:.3f}, MAE={mae_train:.3f}/{mae_test:.3f}, RMSLE={rmsle_train:.3f}/{rmsle_test:.3f}")
            
        except Exception as e:
            print(f"    {m_name} 训练失败: {str(e)[:50]}")
            if CONFIG['debug_mode']:
                import traceback
                print(f"    🔍 详细错误信息: {traceback.format_exc()}")

# ============== 5. 最终模型过滤阶段 ==============
print(f"\n=== 最终模型过滤阶段: 剔除包含指定元素特征的模型 ===")
print(f"满足R²条件(0.905≤R²<0.99)的模型总数: {len(all_qualified_models)}")
print(f"需要剔除的元素特征: {CONFIG['forbidden_features_for_final_model']}")

# 过滤包含禁止元素特征的模型
best_models = []
filtered_models = []

for model_info in all_qualified_models:
    features = model_info['features']
    if isinstance(features, list):
        # 检查是否包含禁止的元素特征
        forbidden_check = [f for f in features if f in CONFIG['forbidden_features_for_final_model']]
        if forbidden_check:
            filtered_models.append({
                'model_key': model_info['model_key'],
                'forbidden_features': forbidden_check,
                'all_features': features,
                'predictions_file': model_info.get('predictions_file')
            })
            print(f"❌ 剔除模型 {model_info['model_key']}: 包含禁止元素特征 {forbidden_check}")
            # 如果有保存预测文件，也记录下来
            if model_info.get('predictions_file'):
                print(f"    📁 预测文件: {os.path.basename(model_info['predictions_file'])}")
        else:
            best_models.append(model_info)
            print(f"✅ 保留模型 {model_info['model_key']}: 不含禁止元素特征")
            # 如果有保存预测文件，也记录下来
            if model_info.get('predictions_file'):
                print(f"    📁 预测文件: {os.path.basename(model_info['predictions_file'])}")
    else:
        # 转换特征方法，无法检查具体特征名，保留
        best_models.append(model_info)
        print(f"✅ 保留模型 {model_info['model_key']}: 特征转换方法")

print(f"\n✓ 最终模型过滤完成:")
print(f"  满足R²条件的模型: {len(all_qualified_models)}个")
print(f"  剔除包含元素特征的模型: {len(filtered_models)}个")
print(f"  最终保留的模型: {len(best_models)}个")

# ============== 6. 显示结果 ==============
print(f"\n=== 模型评估结果 (使用传统+高级特征选择方法+物理引导特征工程+标准验证+默认参数, 最终剔除包含指定元素特征的模型, 无贝叶斯优化) ===")

# 创建特征名称映射
feature_name_mapping = create_feature_name_mapping()

# 保存所有结果到CSV
import csv
csv_file = os.path.join(CONFIG['output_dir'], 'all_model_results_physics_guided_no_bayesian_k4567_fixed.csv')
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Model Key", "K", "Selected Feature Indices", "Selected Feature Names", "Selected Feature Codes",
                    "Train R²", "Test R²", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", 
                    "Train MSE", "Test MSE", "Train RMSLE", "Test RMSLE", "Method Type", "Focus Combination",
                    "Data Type", "Model Instance ID"])
    for model_key, metrics in results_dict.items():
        sel_indices = metrics['Selected Indices']
        if sel_indices is not None:
            sel_indices_str = ', '.join(map(str, sel_indices))
            sel_names = [features_name[i] for i in sel_indices]
            sel_names_str = ', '.join(sel_names)
            # 应用特征名称映射
            sel_codes = apply_feature_name_mapping(sel_names, feature_name_mapping)
            sel_codes_str = ', '.join(sel_codes)
        else:
            sel_indices_str = "N/A (Transformed)"
            sel_names_str = "N/A (Transformed)"
            sel_codes_str = "N/A (Transformed)"
        
        # 判断方法类型
        method_type = "传统方法"
        if any(method in model_key for method in ['SFS_', 'SBS_', 'GA_']):
            method_type = "高级方法"
        
        writer.writerow([
            model_key, metrics['K'], sel_indices_str, sel_names_str, sel_codes_str,
            f"{metrics['Train R²']:.6f}", f"{metrics['Test R²']:.6f}", 
            f"{metrics['Train RMSE']:.6f}", f"{metrics['Test RMSE']:.6f}",
            f"{metrics['Train MAE']:.6f}", f"{metrics['Test MAE']:.6f}",
            f"{metrics['Train MSE']:.6f}", f"{metrics['Test MSE']:.6f}",
            f"{metrics['Train RMSLE']:.6f}", f"{metrics['Test RMSLE']:.6f}",
            method_type, "是" if metrics['Focus Combination'] else "否",
            metrics.get('Data Type', 'Unknown'), metrics.get('Model Instance ID', 'Unknown')
        ])

print(f"✓ 所有结果已保存到: {csv_file}")

# 保存被过滤的模型信息
filtered_csv = os.path.join(CONFIG['output_dir'], 'filtered_models_with_element_features_k4567_fixed.csv')
with open(filtered_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Model Key", "Forbidden Features Found", "All Features", "All Feature Codes", "Predictions File"])
    for model_info in filtered_models:
        # 应用特征名称映射
        feature_codes = apply_feature_name_mapping(model_info['all_features'], feature_name_mapping)
        writer.writerow([
            model_info['model_key'],
            ', '.join(model_info['forbidden_features']),
            ', '.join(model_info['all_features']),
            ', '.join(feature_codes),
            os.path.basename(model_info.get('predictions_file', '')) if model_info.get('predictions_file') else ''
        ])

print(f"✓ 被过滤的模型信息已保存到: {filtered_csv}")

# 保存预测文件索引
predictions_index_file = os.path.join(CONFIG['output_dir'], 'predictions_files_index_k4567.csv')
with open(predictions_index_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Model Key", "Predictions File", "Status", "Features", "Feature Codes", "Test R²", "Test MAE"])
    
    for model_info in all_qualified_models:
        status = "保留" if model_info in best_models else "剔除"
        features_str = ', '.join(model_info['features']) if isinstance(model_info['features'], list) else model_info['features']
        
        # 应用特征名称映射
        if isinstance(model_info['features'], list):
            feature_codes = apply_feature_name_mapping(model_info['features'], feature_name_mapping)
            feature_codes_str = ', '.join(feature_codes)
        else:
            feature_codes_str = model_info['features']
        
        writer.writerow([
            model_info['model_key'],
            os.path.basename(model_info.get('predictions_file', '')) if model_info.get('predictions_file') else '',
            status,
            features_str,
            feature_codes_str,
            f"{model_info['test_r2']:.4f}",
            f"{model_info['test_mae']:.4f}"
        ])

print(f"✓ 预测文件索引已保存到: {predictions_index_file}")

# ============== 7. 重点展示最终保留的模型 ==============
if best_models:
    print(f"\n=== 最终保留的模型 (K=4,5,6,7, 0.905≤Train R²<0.99, 0.905≤Test R²<0.99, 不含指定元素特征, 包含物理引导特征工程+标准验证+默认参数, 无贝叶斯优化) ===")
    print(f"共保留 {len(best_models)} 个模型组合")
    
    # 统计高级方法的表现
    advanced_models = [m for m in best_models if m['is_advanced']]
    focus_models = [m for m in best_models if m['is_focus_model']]
    focus_methods = [m for m in best_models if m['is_focus_method']]
    focus_combinations = [m for m in best_models if m['is_focus_combination']]
    
    print(f"其中 {len(advanced_models)} 个来自高级方法 (SFS, SBS, GA)")
    print(f"其中 {len(focus_models)} 个使用了重点模型 {FOCUS_MODELS}")
    print(f"其中 {len(focus_methods)} 个使用了重点方法 {FOCUS_METHODS}")
    print(f"其中 {len(focus_combinations)} 个是重点组合 (重点模型+重点方法)")
    print(f"注意: 所有模型均使用默认参数，无超参数优化")
    print(f"注意: 当前测试14种机器学习模型")
    print(f"注意: 已剔除包含指定元素特征的模型: {CONFIG['forbidden_features_for_final_model']}")
    print(f"📁 详细预测结果已保存到: {predictions_dir}")
    
    # 保存最终保留的模型
    best_csv = os.path.join(CONFIG['output_dir'], 'best_models_physics_guided_no_bayesian_k4567_fixed.csv')
    with open(best_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["排名", "特征选择方法", "模型", "K", "Train R²", "Test R²", 
                        "Train MAE", "Test MAE", "Train RMSLE", "Test RMSLE", 
                        "选择的特征", "特征简称编码", "方法类型", "重点组合", "数据类型", "预测文件"])
        
        # 按测试集R²排序
        best_models.sort(key=lambda x: x['test_r2'], reverse=True)
        
        for idx, model_info in enumerate(best_models):
            features_str = ', '.join(model_info['features']) if isinstance(model_info['features'], list) else model_info['features']
            
            # 应用特征名称映射
            if isinstance(model_info['features'], list):
                feature_codes = apply_feature_name_mapping(model_info['features'], feature_name_mapping)
                feature_codes_str = ', '.join(feature_codes)
            else:
                feature_codes_str = model_info['features']
                
            method_type = "高级方法" if model_info['is_advanced'] else "传统方法"
            focus_combination_str = "是" if model_info['is_focus_combination'] else "否"
            predictions_file = os.path.basename(model_info.get('predictions_file', '')) if model_info.get('predictions_file') else ''
            
            writer.writerow([
                idx + 1,
                model_info['method'],
                model_info['model'],
                model_info['k'],
                f"{model_info['train_r2']:.4f}",
                f"{model_info['test_r2']:.4f}",
                f"{model_info['train_mae']:.4f}",
                f"{model_info['test_mae']:.4f}",
                f"{model_info['train_rmsle']:.4f}",
                f"{model_info['test_rmsle']:.4f}",
                features_str,
                feature_codes_str,
                method_type,
                focus_combination_str,
                model_info.get('data_type', 'Unknown'),
                predictions_file
            ])
    
    print(f"\n✓ 最终保留的模型已保存到: {best_csv}")
    
    # ============== 打印所有最终保留的模型 ==============
    print(f"\n=== 所有最终保留的模型详细信息 ===")
    for idx, model_info in enumerate(best_models):
        features_str = ', '.join(model_info['features']) if isinstance(model_info['features'], list) else model_info['features']
        
        # 应用特征名称映射
        if isinstance(model_info['features'], list):
            feature_codes = apply_feature_name_mapping(model_info['features'], feature_name_mapping)
            feature_codes_str = ', '.join(feature_codes)
        else:
            feature_codes_str = model_info['features']
        
        advanced_mark = "★" if model_info['is_advanced'] else " "
        focus_mark = "🎯" if model_info['is_focus_combination'] else " "
        
        print(f"\n第{idx+1}名{advanced_mark}{focus_mark}:")
        print(f"  方法: {model_info['method']}, 模型: {model_info['model']}, K={model_info['k']}")
        print(f"  Train R²: {model_info['train_r2']:.4f}, Test R²: {model_info['test_r2']:.4f}")
        print(f"  Train MAE: {model_info['train_mae']:.4f}, Test MAE: {model_info['test_mae']:.4f}")
        print(f"  Train RMSLE: {model_info['train_rmsle']:.4f}, Test RMSLE: {model_info['test_rmsle']:.4f}")
        print(f"  特征: {features_str}")
        print(f"  特征编码: {feature_codes_str}")
        print(f"  数据类型: {model_info.get('data_type', 'Unknown')}")
        
        # 显示预测文件信息
        if model_info.get('predictions_file'):
            filename = os.path.basename(model_info['predictions_file'])
            print(f"  📁 预测文件: {filename}")
        
        # 特殊标记
        marks = []
        if model_info['is_advanced']:
            marks.append("★ 高级方法")
        if model_info['is_focus_combination']:
            marks.append("🎯 重点组合")
        if model_info['is_focus_model']:
            marks.append("🎯 重点模型")
        if model_info['is_focus_method']:
            marks.append("🎯 重点方法")
        marks.append("⚡ 默认参数")
        marks.append("🚫 无指定元素")
        marks.append("💾 预测结果已保存")
        
        if marks:
            print(f"  标记: {' | '.join(marks)}")
    
    # ============== 保存模型和数据 ==============
    print(f"\n=== 保存模型数据 ===")
    
    # 保存模型列表
    models_file = os.path.join(CONFIG['output_dir'], 'best_models_physics_guided_no_bayesian_k4567_fixed.pkl')
    with open(models_file, 'wb') as f:
        pickle.dump(best_models, f)
    print(f"✓ 模型列表已保存到: {models_file}")
    
    # 保存特征名称列表
    features_file = os.path.join(CONFIG['output_dir'], 'features_name_physics_guided_no_bayesian_k4567_fixed.pkl')
    with open(features_file, 'wb') as f:
        pickle.dump(features_name, f)
    print(f"✓ 特征名称已保存到: {features_file}")
    
    # 保存特征名称映射
    mapping_file = os.path.join(CONFIG['output_dir'], 'feature_name_mapping_k4567.pkl')
    with open(mapping_file, 'wb') as f:
        pickle.dump(feature_name_mapping, f)
    print(f"✓ 特征名称映射已保存到: {mapping_file}")
    
    # 保存标准化器
    scaler_file = os.path.join(CONFIG['output_dir'], 'scaler_physics_guided_no_bayesian_k4567_fixed.pkl')
    joblib.dump(scaler, scaler_file)
    print(f"✓ 标准化器已保存到: {scaler_file}")
    
    # 保存其他必要的数据
    data_info = {
        'X_all_scaled': X_all_scaled,
        'X_all_original': X_all_original,
        'y_all_np': y_all_np,
        'initial_features_name': initial_features_name,
        'tree_based_models': tree_based_models,
        'CONFIG': CONFIG,
        'FOCUS_MODELS': FOCUS_MODELS,
        'FOCUS_METHODS': FOCUS_METHODS,
        'forbidden_features_for_final_model': CONFIG['forbidden_features_for_final_model'],
        'advanced_methods_included': True,  # 标记包含高级方法
        'enhanced_metrics_included': True,   # 标记包含增强指标
        'bayesian_optimization_included': False,  # 标记未包含贝叶斯优化
        'physics_guided_enhancement_included': True,  # 标记包含物理引导特征工程
        'default_parameters_only': True,           # 标记仅使用默认参数
        'element_features_filtered_at_end': True,  # 标记在最终阶段过滤元素特征
        'total_qualified_models': len(all_qualified_models),  # 总的满足条件模型数
        'filtered_models_count': len(filtered_models),  # 被过滤的模型数
        'final_models_count': len(best_models),  # 最终保留的模型数
        'detailed_predictions_saved': CONFIG['save_detailed_predictions'],  # 标记是否保存详细预测
        'predictions_directory': predictions_dir,  # 预测文件目录
        'data_consistency_check_enabled': CONFIG['enable_data_consistency_check'],  # 数据一致性检查
        'debug_mode_enabled': CONFIG['debug_mode'],  # 调试模式
        'timestamp': timestamp,  # 时间戳
        'target_k_range': CONFIG['target_k_range'],  # K值范围
        'feature_name_mapping': feature_name_mapping,  # 特征名称映射
        'physics_guided_config': {
            'exact_feature_names': CONFIG['exact_feature_names'],
            'auto_feature_detection': CONFIG['auto_feature_detection'],
            'quality_check_enabled': CONFIG['quality_check_enabled'],
            'correlation_threshold': CONFIG['correlation_threshold'],
            'variance_threshold': CONFIG['variance_threshold']
        }
    }
    data_file = os.path.join(CONFIG['output_dir'], 'data_info_physics_guided_no_bayesian_k4567_fixed.pkl')
    with open(data_file, 'wb') as f:
        pickle.dump(data_info, f)
    print(f"✓ 数据信息已保存到: {data_file}")
    
else:
    print(f"\n⚠️ 没有最终保留的模型 (经过元素特征过滤后)")
    print("可能的原因:")
    print("1. 满足R²条件的模型都包含了指定的元素特征")
    print("2. R²阈值设置过严格")
    print("3. 指定的禁止元素特征过多")
    print(f"   满足R²条件的模型总数: {len(all_qualified_models)}")
    print(f"   被过滤的模型数: {len(filtered_models)}")
    print(f"   禁止的元素特征: {CONFIG['forbidden_features_for_final_model']}")

# ============== 8. 统计分析 ==============
print(f"\n=== 统计分析 (最终模型过滤策略: 在模型训练完成后剔除包含指定元素特征的模型) ===")

print(f"\n模型过滤统计:")
print(f"  满足R²条件(0.905≤R²<0.99)的模型总数: {len(all_qualified_models)}")
print(f"  包含指定元素特征被剔除的模型数: {len(filtered_models)}")
print(f"  最终保留的模型数: {len(best_models)}")
print(f"  模型保留率: {len(best_models)/len(all_qualified_models)*100:.1f}%" if all_qualified_models else "0%")

if filtered_models:
    print(f"\n被剔除模型的元素特征统计:")
    element_count = {}
    for model_info in filtered_models:
        for element in model_info['forbidden_features']:
            element_count[element] = element_count.get(element, 0) + 1
    
    sorted_elements = sorted(element_count.items(), key=lambda x: x[1], reverse=True)
    for element, count in sorted_elements:
        print(f"  {element}: {count}个模型包含")

if best_models:
    print(f"\n最终保留模型的性能统计:")
    final_train_r2 = [m['train_r2'] for m in best_models]
    final_test_r2 = [m['test_r2'] for m in best_models]
    final_train_mae = [m['train_mae'] for m in best_models]
    final_test_mae = [m['test_mae'] for m in best_models]
    
    print(f"  训练集R² - 平均: {np.mean(final_train_r2):.4f}, 最大: {np.max(final_train_r2):.4f}, 最小: {np.min(final_train_r2):.4f}")
    print(f"  测试集R² - 平均: {np.mean(final_test_r2):.4f}, 最大: {np.max(final_test_r2):.4f}, 最小: {np.min(final_test_r2):.4f}")
    print(f"  训练集MAE - 平均: {np.mean(final_train_mae):.4f}, 最大: {np.max(final_train_mae):.4f}, 最小: {np.min(final_train_mae):.4f}")
    print(f"  测试集MAE - 平均: {np.mean(final_test_mae):.4f}, 最大: {np.max(final_test_mae):.4f}, 最小: {np.min(final_test_mae):.4f}")
    
    # 按K值统计
    k_stats = {}
    for model in best_models:
        k = model['k']
        if k not in k_stats:
            k_stats[k] = []
        k_stats[k].append(model['test_r2'])
    
    print(f"\n按特征数K统计:")
    for k in sorted(k_stats.keys()):
        r2_values = k_stats[k]
        print(f"  K={k}: {len(r2_values)}个模型, 平均R²={np.mean(r2_values):.4f}, 最佳R²={np.max(r2_values):.4f}")

# ============== 9. 找出最优模型 ==============
if best_models:
    best_model_info = max(best_models, key=lambda x: x['test_r2'])
    
    print(f"\n=== 全局最优结果 (经过元素特征过滤后) ===")
    print(f"最优模型: {best_model_info['model_key']}")
    print(f"测试集R²: {best_model_info['test_r2']:.4f}")
    print(f"测试集MAE: {best_model_info['test_mae']:.4f}")
    print(f"测试集RMSLE: {best_model_info['test_rmsle']:.4f}")
    print(f"特征选择方法: {best_model_info['method']}")
    print(f"机器学习模型: {best_model_info['model']}")
    
    features_for_display = best_model_info['features'] if isinstance(best_model_info['features'], list) else [best_model_info['features']]
    print(f"选择的特征: {', '.join(features_for_display)}")
    
    # 显示特征编码
    if isinstance(best_model_info['features'], list):
        feature_codes = apply_feature_name_mapping(best_model_info['features'], feature_name_mapping)
        print(f"特征编码: {', '.join(feature_codes)}")
    
    print(f"使用的数据类型: {best_model_info.get('data_type', 'Unknown')}")
    
    if best_model_info.get('predictions_file'):
        print(f"📁 详细预测结果文件: {os.path.basename(best_model_info['predictions_file'])}")
    
    marks = []
    if best_model_info['is_advanced']:
        marks.append("★ 高级方法")
    if best_model_info['is_focus_combination']:
        marks.append("🎯 重点组合")
    marks.append("⚡ 默认参数")
    marks.append("🚫 无指定元素")
    marks.append("💾 预测结果已保存")
    
    if marks:
        print(f"特殊标记: {' | '.join(marks)}")
    
    # 保存最优模型信息
    best_info = {
        'best_model': best_model_info['model_key'],
        'best_r2': best_model_info['test_r2'],
        'best_mae': best_model_info['test_mae'],
        'best_rmsle': best_model_info['test_rmsle'],
        'best_method': best_model_info['method'],
        'best_ml_model': best_model_info['model'],
        'best_features': best_model_info['features'],
        'best_feature_codes': apply_feature_name_mapping(best_model_info['features'], feature_name_mapping) if isinstance(best_model_info['features'], list) else best_model_info['features'],
        'best_is_advanced': best_model_info['is_advanced'],
        'best_is_focus_combination': best_model_info['is_focus_combination'],
        'best_data_type': best_model_info.get('data_type', 'Unknown'),
        'best_predictions_file': best_model_info.get('predictions_file'),
        'total_feature_sets': len(good_feature_sets),
        'total_models_tested': len(results_dict),
        'total_qualified_models': len(all_qualified_models),
        'filtered_models': len(filtered_models),
        'final_models': len(best_models),
        'forbidden_features': CONFIG['forbidden_features_for_final_model'],
        'physics_guided_enhancement': CONFIG['physics_guided_enhancement'],
        'bayesian_optimization': False,
        'default_parameters_only': True,
        'element_features_filtered_at_end': True,
        'detailed_predictions_saved': CONFIG['save_detailed_predictions'],
        'data_consistency_check_enabled': CONFIG['enable_data_consistency_check'],
        'debug_mode_enabled': CONFIG['debug_mode'],
        'min_r2_threshold': CONFIG['min_r2_threshold'],
        'max_r2_threshold': CONFIG['max_r2_threshold'],
        'target_k_range': CONFIG['target_k_range'],
        'timestamp': timestamp
    }
    
    best_file = os.path.join(CONFIG['output_dir'], 'best_model_info_physics_guided_no_bayesian_k4567_fixed.txt')
    with open(best_file, 'w', encoding='utf-8') as f:
        f.write("=== 模型训练总结 (修复版: 传统+高级特征选择方法+物理引导特征工程+标准验证+默认参数, K=4,5,6,7, 最终剔除包含指定元素特征的模型, 无贝叶斯优化) ===\n")
        f.write(f"最优模型: {best_info['best_model']}\n")
        f.write(f"最优测试集R²: {best_info['best_r2']:.4f}\n")
        f.write(f"最优测试集MAE: {best_info['best_mae']:.4f}\n")
        f.write(f"最优测试集RMSLE: {best_info['best_rmsle']:.4f}\n")
        f.write(f"最优特征选择方法: {best_info['best_method']}\n")
        f.write(f"最优机器学习模型: {best_info['best_ml_model']}\n")
        
        features_for_txt = best_info['best_features'] if isinstance(best_info['best_features'], list) else [best_info['best_features']]
        f.write(f"最优模型特征: {', '.join(features_for_txt)}\n")
        
        codes_for_txt = best_info['best_feature_codes'] if isinstance(best_info['best_feature_codes'], list) else [best_info['best_feature_codes']]
        f.write(f"最优模型特征编码: {', '.join(codes_for_txt)}\n")
        
        f.write(f"最优模型使用的数据类型: {best_info['best_data_type']}\n")
        f.write(f"最优模型预测文件: {os.path.basename(best_info['best_predictions_file']) if best_info['best_predictions_file'] else 'None'}\n")
        f.write(f"最优模型是否来自高级方法: {'是' if best_info['best_is_advanced'] else '否'}\n")
        f.write(f"最优模型是否是重点组合: {'是' if best_info['best_is_focus_combination'] else '否'}\n")
        f.write(f"使用贝叶斯优化: {'是' if best_info['bayesian_optimization'] else '否'}\n")
        f.write(f"仅使用默认参数: {'是' if best_info['default_parameters_only'] else '否'}\n")
        f.write(f"最终阶段过滤元素特征: {'是' if best_info['element_features_filtered_at_end'] else '否'}\n")
        f.write(f"保存详细预测结果: {'是' if best_info['detailed_predictions_saved'] else '否'}\n")
        f.write(f"数据一致性检查: {'启用' if best_info['data_consistency_check_enabled'] else '禁用'}\n")
        f.write(f"调试模式: {'启用' if best_info['debug_mode_enabled'] else '禁用'}\n")
        f.write(f"特征数范围: K={best_info['target_k_range']}\n")
        f.write(f"程序运行时间戳: {best_info['timestamp']}\n")
        f.write(f"\n=== 模型统计 ===\n")
        f.write(f"评估的特征集总数: {best_info['total_feature_sets']}\n")
        f.write(f"训练的模型总数: {best_info['total_models_tested']}\n")
        f.write(f"满足R²条件(0.905≤R²<0.99)的模型总数: {best_info['total_qualified_models']}\n")
        f.write(f"包含指定元素特征被剔除的模型数: {best_info['filtered_models']}\n")
        f.write(f"最终保留的模型数: {best_info['final_models']}\n")
        f.write(f"模型保留率: {best_info['final_models']/best_info['total_qualified_models']*100:.1f}%\n")
        f.write(f"R²阈值范围: {best_info['min_r2_threshold']:.3f} ≤ R² < {best_info['max_r2_threshold']:.2f} (避免过拟合)\n")
        f.write(f"最终剔除的指定元素特征: {best_info['forbidden_features']}\n")
        f.write(f"物理引导特征工程: {'启用' if best_info['physics_guided_enhancement'] else '禁用'}\n")
        f.write(f"\n=== 修复内容 ===\n")
        f.write(f"1. 修改target_k_range为[4,5,6,7]，扩展特征数范围\n")
        f.write(f"2. 添加特征名称简称编码映射，基于PDF文档的139个特征编码系统\n")
        f.write(f"3. 修复CSV文件保存时的编码错误问题\n")
        f.write(f"4. 所有输出文件名添加k4567标识，区分不同K值范围的结果\n")
        f.write(f"5. 在结果显示中同时显示原特征名和简称编码\n")
        f.write(f"6. 保存特征名称映射字典，便于后续使用\n")
        f.write(f"\n=== 特征编码系统 ===\n")
        f.write(f"成分特征(CC): 12个元素成分特征，CC1-CC12\n")
        f.write(f"原子电子(AE): 10个原子电子性质特征，AE1m-AE5v\n")
        f.write(f"原子半径(AR): 原子半径相关特征\n")
        f.write(f"原子磁性(AM): 磁化率相关特征\n")
        f.write(f"原子核(AN): 原子质量相关特征\n")
        f.write(f"原子重量(AW): 原子重量相关特征\n")
        f.write(f"热力学(TH): 8个热力学特征\n")
        f.write(f"机械性能(PM): 6个机械性能特征\n")
        f.write(f"热学性能(PT): 18个热学性能特征\n")
        f.write(f"电学性能(PE): 4个电学性能特征\n")
        f.write(f"密度性能(PD): 4个密度性能特征\n")
        f.write(f"晶格结构(SL): 6个晶格结构特征\n")
        f.write(f"空间群(SS): 2个空间群特征\n")
        f.write(f"物理工程特征(36个): JT韧性判据5个, JE电子结构4个, JH热力学4个, JN-NbSi专用5个, JM力学5个, JD扩散2个, JA合金3个, JS结构5个, JX其他3个\n")
        f.write(f"\n=== 技术配置 ===\n")
        f.write(f"特征数: K=4,5,6,7 (从原来的K=4,5扩展)\n")
        f.write(f"传统特征选择方法: 8种 (RFE, 互信息, F回归, PCC, Lasso, 随机森林, ANOVA, SelectKBest)\n")
        f.write(f"高级特征选择方法: 3种 (SFS, SBS, GA)\n")
        f.write(f"SFS配置: Ridge回归评估，最大{CONFIG['sfs_max_iter']}次迭代\n")
        f.write(f"SBS配置: Ridge回归评估，最大{CONFIG['sbs_max_iter']}次迭代\n")
        f.write(f"GA配置: 种群{CONFIG['ga_population']}，代数{CONFIG['ga_generations']}，变异率{CONFIG['ga_mutation_rate']}\n")
        f.write(f"物理引导特征工程配置: 基于实际数据库特征的34个物理意义特征\n")
        f.write(f"机器学习模型: 16种\n")
        f.write(f"所有模型均使用默认参数，无超参数优化\n")
        f.write(f"评估指标: R², RMSE, MAE, MSE, RMSLE (训练集和测试集)\n")
        f.write(f"验证策略: 标准80%-20%分割验证\n")
        f.write(f"重点模型: {FOCUS_MODELS}\n")
        f.write(f"重点方法: {FOCUS_METHODS}\n")
        f.write(f"特征筛选策略: 传统统计方法+高级搜索算法+物理引导特征工程\n")
        f.write(f"评估策略: 高级方法内部使用Ridge回归3折交叉验证\n")
        f.write(f"超参数优化策略: 无超参数优化，所有模型使用默认参数\n")
        f.write(f"元素特征过滤策略: 在模型训练完成后剔除包含指定元素特征的模型\n")
        f.write(f"元素特征限制: 剔除包含{len(CONFIG['forbidden_features_for_final_model'])}个元素特征的模型: {CONFIG['forbidden_features_for_final_model']}\n")
        f.write(f"递归消除配置: max_k={CONFIG['max_k']}，使用Ridge、Huber、随机森林组合评估\n")
        f.write(f"验证策略: 标准80%-20%训练测试分割\n")
        f.write(f"并行设置: 使用 {CONFIG['n_jobs']} 个进程进行并行计算\n")
        f.write(f"方法创新: 结合传统统计方法、现代搜索算法、物理引导特征工程，全面探索特征空间\n")
        f.write(f"增强功能: 新增MAE、MSE、RMSLE等评估指标，提供更全面的模型性能评估\n")
        f.write(f"物理引导特征工程: 基于您实际数据库特征的34个物理意义明确的特征，涵盖韧性判据、电子结构、热力学等9大类别\n")
        f.write(f"性能优化: 移除贝叶斯优化，大幅减少计算时间，适合快速原型开发和基准测试\n")
        f.write(f"质量控制: 最终阶段的元素特征过滤，确保保存的模型不依赖特定的元素成分\n")
        f.write(f"数据完整性: 每个满足条件的模型都有对应的详细预测结果文件，确保结果可追溯\n")
        f.write(f"编码系统: 基于PDF文档的139个特征完整编码系统，提供简洁的特征标识\n")
    
    print(f"✓ 最优模型信息已保存到: {best_file}")

print(f"\n=== 程序执行完成 (修复版 K=4,5,6,7) ===")
print(f"输出目录: {CONFIG['output_dir']}")
print(f"特征数范围: K={CONFIG['target_k_range']}")
print(f"最终剔除包含以下元素特征的模型: {CONFIG['forbidden_features_for_final_model']}")
print(f"R²阈值范围: {CONFIG['min_r2_threshold']:.2f} ≤ R² < {CONFIG['max_r2_threshold']:.2f} (避免过拟合)")
print(f"程序运行时间戳: {timestamp}")
print(f"\n请查看以下文件:")
print(f"1. all_model_results_physics_guided_no_bayesian_k4567_fixed.csv - 所有模型结果(含特征编码)")
print(f"2. best_models_physics_guided_no_bayesian_k4567_fixed.csv - 最终保留的模型(含特征编码)")
print(f"3. filtered_models_with_element_features_k4567_fixed.csv - 被剔除的模型(含特征编码)")
print(f"4. predictions_files_index_k4567.csv - 预测文件索引(含特征编码)")
print(f"5. best_model_info_physics_guided_no_bayesian_k4567_fixed.txt - 最优模型总结")
print(f"6. best_models_physics_guided_no_bayesian_k4567_fixed.pkl - 保存的模型实例")
print(f"7. features_name_physics_guided_no_bayesian_k4567_fixed.pkl - 特征名称")
print(f"8. feature_name_mapping_k4567.pkl - 特征名称映射字典")
print(f"9. scaler_physics_guided_no_bayesian_k4567_fixed.pkl - 标准化器")
print(f"10. data_info_physics_guided_no_bayesian_k4567_fixed.pkl - 数据信息")
print(f"11. detailed_predictions/ - 详细预测结果目录")
print(f"\n✅ 主要修改内容:")
print(f"🔧 K值范围: 从[4,5]扩展到[4,5,6,7]，增加特征选择的灵活性")
print(f"🔧 特征编码: 基于PDF文档的139个特征编码系统，提供简洁标识")
print(f"🔧 编码错误修复: 解决CSV文件保存时的字符编码问题")
print(f"🔧 文件命名: 所有输出文件添加k4567标识，便于识别")
print(f"🔧 显示优化: 结果中同时显示完整特征名和简称编码")
print(f"🔧 映射保存: 单独保存特征名称映射字典，便于后续使用")
print(f"\n现在您可以:")
print(f"• 测试K=4,5,6,7四种不同的特征数组合")
print(f"• 使用简洁的特征编码(如CC1, AE2m, JT1等)快速识别特征")
print(f"• 根据编码系统快速了解特征的物理意义和类别")
print(f"• 避免长特征名造成的显示和分析困难")
print(f"• 使用标准化的特征命名体系进行后续研究")

检测到 12 个CPU核心，将使用并行计算
=== 开始数据加载与预处理 ===
✓ 成功加载数据: (221, 104)
✓ 动态识别到 103 个数值特征
✓ 目标列: KQ
✓ 将在最终模型保存时过滤的元素特征: ['Nb', 'Si', 'Ti', 'Al', 'Cr', 'Hf', 'Zr', 'Mo', 'V', 'W', 'Ta', 'Sn']

=== 数据标准化与清理 ===
✓ 移除NaN后数据形状: X(221, 103), y(221,)
✓ 交叉验证设置: 仅在特征筛选阶段使用5折交叉验证

=== 数据一致性检查 ===
    🔍 标准化特征矩阵 数据校验和: e62c74df
    🔍 目标变量 数据校验和: c550c047

=== 物理引导特征工程阶段 ===
=== 基于实际数据库的物理引导特征工程 ===
  基础特征数: 103
  数据库特征验证: 检查必要特征是否存在

  === 核心韧性判据特征 ===
  ✅ Pugh模量比 (B/G) - 基于VEC计算
  ✅ 预测泊松比 - 基于VEC计算
  ✅ Cauchy压力指标 - 基于VEC计算
  ✅ Si过饱和度 - 基于Si含量计算
  ✅ Nb-Si原子尺寸失配 - 基于Nb、Si含量计算

  === 电子结构特征 ===
  ✅ VEC-Ω耦合
  ✅ VEC稳定化效应
  ✅ VEC偏离度特征
  ✅ 电子局域化参数

  === 热力学稳定性特征 ===
  ✅ 热力学驱动力
  ✅ 熵稳定化效应
  ✅ 焓熵竞争效应
  ✅ 相稳定性指数
  ✅ 热稳定性因子

  === Nb-Si体系专用特征 ===
  ✅ Nb/Si比值
  ✅ 共晶距离和过共晶指示器
  ✅ 硅化物形成倾向
  ✅ 硅化物体积分数预测
  ✅ 固溶度限制因子

  === 韧性和力学性能特征 ===
  ✅ 韧性指示器
  ✅ 裂纹阻力因子
  ✅ 弹性各向异性参数
  ✅ 塑性变形能力指标
  ✅ 微观应力集中因子

  === 扩散和动力学特征 ===
  ✅ 扩散阻力
  ✅ 扩散激活能估算

  === 合金元素协同效应 ===
  ✅ 难熔元素强化效应 (包含6个元素: ['Ti', 'Hf', 'Zr', 'Mo', 'V', 'Ta'])
  ✅ Ti-

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   26.5s
[Parallel(n_jobs=4)]: Done  92 out of  92 | elapsed:   54.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


  第 1 轮: 移除 'silicide_volume_fraction', 误差: 87.7035

  第 2 轮评估 (剩余特征: 91)


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   26.0s
[Parallel(n_jobs=4)]: Done  91 out of  91 | elapsed:   54.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


  第 2 轮: 移除 'lattice_mismatch_effect', 误差: 49.2062

  第 3 轮评估 (剩余特征: 90)


...

  第 50 轮: 移除 'var_共价半径 Pyykko(Triple Bond) (pm)', 误差: 41.0086

  第 51 轮评估 (剩余特征: 42)


[Parallel(n_jobs=4)]: Done  42 out of  42 | elapsed:   16.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


  第 51 轮: 移除 'mean_热膨胀(1/k)', 误差: 41.0009

  第 52 轮评估 (剩余特征: 41)


[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:   14.3s finished


  第 52 轮: 移除 'ΔTm', 误差: 41.1603
  达到最大特征数限制 (K=40)，停止递归消除
✓ 第二阶段完成，剩余 40 个特征
    🔍 递归消除后的特征矩阵 数据校验和: fc3919b8

=== 开始特征筛选 (处理K=4,5,6,7, 使用传统+高级特征选择方法+物理引导特征工程) ===
传统方法: 8种 (RFE, 互信息, F回归, PCC, Lasso, 随机森林, ANOVA, SelectKBest)
高级方法: 3种 (SFS, SBS, GA)
物理引导: 基于实际数据库特征的物理意义特征工程
筛选策略: 各方法直接选择最优特征，无需额外评估
注意: 元素特征将参与特征选择，但包含指定元素特征的模型在最终保存时将被剔除

--- 特征数 K = 4 ---
    🔍 RFE_k=4 选择的特征矩阵 数据校验和: ff694ae2
                      RFE | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 Mutual Information_k=4 选择的特征矩阵 数据校验和: f6fe3711
       Mutual Information | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 F-Regression_k=4 选择的特征矩阵 数据校验和: 41105d23
             F-Regression | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 PCC_k=4 选择的特征矩阵 数据校验和: fd5074c3
                      PCC | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 Lasso_k=4 选择的特征矩阵 数据校验和: a266ff2e
                    Lasso | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 RandomForest_k=4 选择的特征矩阵 数据校验和: f459a402
             RandomForest | 成功选择 4 个特征
    ✓ 特征集已加入
    🔍 ANOVA_F_k=4 选择的特征矩阵 数据校验和: 681acd0e
                  ANOVA_